In [ ]:
! pip install imdbPY

In [ ]:
import imdb
import csv
import pandas as pd
from imdb import IMDb, IMDbDataAccessError

In [ ]:
ia = imdb.IMDb()
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

In [ ]:
data=pd.read_csv(r'/kaggle/input/movielens/movies.csv')

In [ ]:
#old verison 
import imdb
import csv
import random
import pandas as pd
import concurrent.futures

# Load the movie data
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        data = {
            'film_name': movie.get('title', ''),
            'film_rate': movie.get('rating', 0.0),  # default to 0.0 if no rating
            'plot': movie.get('plot', ''),
            'cover_url': movie.get('cover url', ''),
            'cast': ', '.join([person['name'] for person in movie.get('cast', [])[:3]]) if movie.get('cast') else '',  # return empty string if cast is not available
            'reviews': movie.get('reviews', [{}])[0].get('content', '')  # default to empty string if no reviews
        }
        return data
    except IMDbDataAccessError as e:
        return {'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': ''}  # return default values
    except urllib.error.HTTPError as e:
        return {'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': ''}  # return default values

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())
csv_file='/kaggle/input/imdbdata/imdb_data.csv'
# Save the processed data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['film_name', 'film_rate', 'plot', 'cover_url', 'cast', 'reviews'])
    for data in data_list:
        writer.writerow(list(data.values()))

# updated version with more features 

In [ ]:
import imdb
import csv
import random
import pandas as pd
import concurrent.futures
import urllib.error


# Load the movie data
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        ia.update(movie, 'directors')
        ia.update(movie, 'writers')

        data = {
            'imdbId': movie_id,
            'film_name': movie.get('title', ''),
            'film_rate': movie.get('rating', 0.0),  # default to 0.0 if no rating
            'plot': movie.get('plot', [''])[0],  # take the first plot synopsis
            'cover_url': movie.get('cover url', ''),
            'cast': ', '.join([person['name'] for person in movie.get('cast', [])[:3]]) if movie.get('cast') else '',  # top 3 cast
            'reviews': movie.get('reviews', [{}])[0].get('content', ''),  # first review content
            'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'writer': ', '.join([person.get('name', '') for person in movie.get('writers', [])]) if movie.get('writers') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''  # join genre list
        }
        return data
    except imdb.IMDbDataAccessError as e:
        return {'imdbId': movie_id, 'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': '', 'director': '', 'writer': '', 'genre': ''}
    except urllib.error.HTTPError as e:
        return {'imdbId': movie_id, 'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': '', 'director': '', 'writer': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'film_rate', 'plot', 'cover_url', 'cast', 'reviews', 'director', 'writer', 'genre'])
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['film_rate'], data['plot'], data['cover_url'], data['cast'], data['reviews'], data['director'], data['writer'], data['genre']])


# missing features only 

In [ ]:
import imdb
import csv
import pandas as pd
import concurrent.futures
import urllib.error

imdbs = data['imdbId']

ia = imdb.IMDb()

batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, info=['directors', 'genres'])  # Only updating required fields

        data = {
            'imdbId': movie_id,
            'film_name': movie.get('title', ''),
            'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''
        }
        return data
    except imdb.IMDbDataAccessError as e:
        # Handle IMDb Data Access Error
        return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}
    except urllib.error.HTTPError as e:
        # Handle HTTP Error
        return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


In [ ]:
# Specify the path to save the CSV in the writable /kaggle/working directory
csv_file = '/kaggle/working/imdb_data.csv'  # Change the directory to /kaggle/working

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


# handling data more effectivly

In [ ]:
import imdb
import csv
import pandas as pd
import concurrent.futures
import urllib.error
import time
from tqdm import tqdm  # For progress tracking

# Load the movie IDs (Assuming 'data' is a DataFrame that contains imdbId)
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data with retry logic
def get_movie_data(movie_id, retries=3):
    for _ in range(retries):
        try:
            movie = ia.get_movie(movie_id)

            # Fetch the required data without explicitly updating 'directors' or 'genres'
            data = {
                'imdbId': movie_id,
                'film_name': movie.get('title', ''),
                'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
                'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''
            }
            return data
        except (imdb.IMDbDataAccessError, urllib.error.HTTPError) as e:
            time.sleep(1)  # Wait for 1 second before retrying
    # If retries are exhausted, return empty data
    return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel and use tqdm for progress tracking
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results with progress tracking
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing batches"):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


# updated needed features to current file

In [ ]:
import imdb
import pandas as pd
import concurrent.futures

# Extract the list of IMDb IDs
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        ia.update(movie, 'directors')
        ia.update(movie, 'writers')

        # Return the new data, using `.get()` to safely handle missing fields
        data = {
            'imdbId': movie_id,
            'director': ', '.join([person.get('name', '') for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'writer': ', '.join([person.get('name', '') for person in movie.get('writers', [])]) if movie.get('writers') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''  # join genre list
        }
        return data
    except imdb.IMDbDataAccessError as e:
        return {'imdbId': movie_id, 'director': '', 'writer': '', 'genre': ''}
    
# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Convert the collected data into a DataFrame
new_data_df = pd.DataFrame(data_list)

# Merge the new data with the existing dataset
updated_data = pd.merge(data, new_data_df, on='imdbId', how='left')

# Save the updated data back to the CSV file
updated_data.to_csv(csv_file, index=False)


In [ ]:
data_list

# download the csv file

In [ ]:
import pandas as pd

# Create a Pandas DataFrame from the data_list
df = pd.DataFrame(data_list)

df.to_csv('imdb_data.csv', index=False)
from IPython.display import FileLink

# This will create a download link for the file
FileLink('imdb_data.csv')

# imdb reccomendation

In [ ]:
import imdb
import pandas as pd
import concurrent.futures

# Set up the IMDb API
ia = imdb.IMDb()

# Define the function to get movie data based on movie title or IMDb ID
def get_movie_id(user_input):
    # Try to interpret input as an IMDb ID (numeric)
    try:
        movie_id = int(user_input)
        movie = ia.get_movie(movie_id)
    except ValueError:
        # If not a number, assume it's a movie title and search by title
        search_results = ia.search_movie(user_input)
        if search_results:
            # Take the first search result as the most likely match
            movie = search_results[0]
        else:
            return None  # Return None if no results found
    
    return movie.movieID  # Return the movie ID

# Define the function to get movie recommendations
def get_recommendations(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'recommendations')  # Fetch recommendations

        # Get the recommended movies (up to 3 recommendations)
        recommended_movies = movie.get('recommendations', [])[:3]
        recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies]) if recommended_movies else 'No recommendations'

        # Return the top 3 recommendations
        return {
            'imdbId': movie_id,
            'recommended_movies': recommended_titles  # Add the top 3 recommendations
        }
    except imdb.IMDbDataAccessError:
        return {'imdbId': movie_id, 'recommended_movies': 'No recommendations'}

# User input function to accept either movie title or IMDb ID
def get_user_recommendations():
    user_input = input("Enter a movie name or IMDb ID: ")

    # Normalize user input (case insensitive, remove extra spaces)
    user_input_normalized = user_input.strip().lower()

    # Get the IMDb movie ID
    movie_id = get_movie_id(user_input_normalized)
    if movie_id:
        # Fetch recommendations if the movie ID is valid
        recommendations = get_recommendations(movie_id)
        print(f"Top 3 recommended movies for {user_input}:")
        print(recommendations['recommended_movies'])
    else:
        print("Sorry, no movie found with that name or IMDb ID.")

# Run the recommendation system for user input
get_user_recommendations()


In [ ]:
get_user_recommendations()

In [ ]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations
def get_recommendations(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)

        # Try to update with recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])

        # If recommendations are missing, provide more feedback
        if not recommended_movies:
            return f"No recommendations available for '{movie_title}'. The IMDb API may not have this data."

        # Format the top 3 recommendations
        recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
        return f"Top 3 recommended movies for {movie_title}:\n{recommended_titles}"

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display recommendations
    result = get_recommendations(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Access genres and keywords directly from the movie object
        genres = movie.get('genres', [])
        keywords = movie.get('keywords', [])
        print(f"Genres: {genres}")
        print(f"Keywords: {keywords}")

        # Search for similar movies based on genre or keywords
        if genres:
            similar_titles = ', '.join([f"Similar movie {i+1}" for i in range(3)])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        if keywords:
            similar_titles = ', '.join([f"Similar movie {i+1}" for i in range(3)])
            return f"No recommendations available for '{movie_title}'. Similar movies by keyword:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        keywords = movie.get('keywords', [])
        print(f"Genres: {genres}")
        print(f"Keywords: {keywords}")

        # Search for similar movies based on genre or keywords
        if genres:
            similar_titles = ', '.join([f"Similar movie {i+1}" for i in range(3)])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        if keywords:
            similar_titles = ', '.join([f"Similar movie {i+1}" for i in range(3)])
            return f"No recommendations available for '{movie_title}'. Similar movies by keyword:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        keywords = movie.get('keywords', [])
        print(f"Genres: {genres}")
        print(f"Keywords: {keywords}")

        # Search for similar movies based on genre
        if genres:
            similar_movies = []
            for genre in genres:
                genre_movies = ia.get_top50_movies_by_genres(genre)
                similar_movies.extend(genre_movies)
            similar_movies = list({movie.movieID: movie for movie in similar_movies}.values())  # Remove duplicates
            similar_titles = ', '.join([sim_movie.get('title', '') for sim_movie in similar_movies[:3]])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        print(f"Genres: {genres}")

        # Search for similar movies based on genre
        if genres:
            similar_movies = []
            for genre in genres:
                search_results = ia.search_movie_advanced(genre=genre)
                similar_movies.extend(search_results[:3])  # Get top 3 movies for each genre
            similar_movies = list({movie.movieID: movie for movie in similar_movies}.values())  # Remove duplicates
            similar_titles = ', '.join([sim_movie.get('title', '') for sim_movie in similar_movies[:3]])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        print(f"Genres: {genres}")

        # Search for similar movies based on genre
        if genres:
            similar_movies = []
            for genre in genres:
                search_results = ia.search_movie(genre)
                similar_movies.extend(search_results[:3])  # Get top 3 movies for each genre
            similar_movies = list({movie.movieID: movie for movie in similar_movies}.values())  # Remove duplicates
            similar_titles = ', '.join([sim_movie.get('title', '') for sim_movie in similar_movies[:3]])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        print(f"Genres: {genres}")

        # Search for similar movies based on all genres
        similar_movies = []
        for genre in genres:
            search_results = ia.search_movie(f"{genre} movie")
            similar_movies.extend(search_results[:3])  # Get top 3 movies for each genre
        similar_movies = list({movie.movieID: movie for movie in similar_movies}.values())  # Remove duplicates
        similar_titles = ', '.join([sim_movie.get('title', '') for sim_movie in similar_movies[:3]])

        return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to fetch basic movie details
def get_movie_info(movie_input):
    try:
        # Check if input is IMDb ID (numeric) or movie title (string)
        if movie_input.isdigit():
            movie = ia.get_movie(movie_input)
        else:
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result
        
        # Print basic information for debugging
        movie_title = movie.get('title', 'Unknown Title')
        movie_year = movie.get('year', 'Unknown Year')
        genres = movie.get('genres', [])
        print(f"Movie found: {movie_title} ({movie_year})")
        print(f"Genres: {genres}")

        return f"Movie: {movie_title} ({movie_year}) | Genres: {', '.join(genres)}"

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display movie info
    result = get_movie_info(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
import imdb
import logging
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

# Set up logging
logging.basicConfig(level=logging.INFO)

# Set up the IMDb API
ia = imdb.IMDb()

def get_movie(movie_title):
    """
    Fetch movie data from IMDb.

    Args:
        movie_title (str): Movie title.

    Returns:
        dict: Movie data.
    """
    try:
        search_results = ia.search_movie(movie_title)
        if search_results:
            movie_id = search_results[0].movieID
            movie = ia.get_movie(movie_id)
            return movie
        else:
            return None
    except imdb.IMDbDataAccessError as e:
        logging.error(f"Error accessing IMDb data: {e}")
        return None
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None

def get_similar_movies(movie):
    """
    Find similar movies based on genre and director.

    Args:
        movie (dict): Movie data.

    Returns:
        list: Similar movies.
    """
    genres = movie.data.get('genres', [])
    directors = movie.data.get('directors', [])

    # Create a list of all unique genres
    all_genres = set()
    for genre in genres:
        all_genres.add(genre)

    # Get top 250 movies
    top_movies = ia.get_top250_movies()

    # Create a list of all unique genres for top movies
    top_movies_data = []
    for top_movie in top_movies:
        top_movie_id = top_movie.movieID
        top_movie_data = ia.get_movie(top_movie_id)
        top_movies_data.append(top_movie_data)
        top_movie_genres = top_movie_data.data.get('genres', [])

        for genre in top_movie_genres:
            all_genres.add(genre)

    # Create a one-hot encoder for genres
    encoder = OneHotEncoder(handle_unknown='ignore')
    all_genres = list(all_genres)
    encoder.fit([[genre] for genre in all_genres])

    # Create a matrix of movie attributes
    movie_attributes = encoder.transform([[genre] for genre in genres]).toarray()

    # Create a matrix of top movie attributes
    top_movie_attributes = []
    for top_movie_data in top_movies_data:
        top_movie_genres = top_movie_data.data.get('genres', [])

        if top_movie_genres:  # Check if top_movie_genres is not empty
            top_movie_attributes_row = encoder.transform([[genre] for genre in top_movie_genres]).toarray()
            top_movie_attributes.append(top_movie_attributes_row)
        else:
            # If top movie has no genres, use a zero vector as its attributes
            top_movie_attributes_row = np.zeros((1, len(all_genres)))
            top_movie_attributes.append(top_movie_attributes_row)

    # Check if top_movie_attributes is not empty
    if top_movie_attributes:
        # Stack the top movie attributes into a single 2D array
        top_movie_attributes = np.vstack(top_movie_attributes)

        # Calculate the cosine similarity between the movie attributes and top movie attributes
        similarity_matrix = cosine_similarity(movie_attributes, top_movie_attributes)

        # Get the top N similar movies
        N = 10
        similar_movies = []
        if similarity_matrix.size:  # Check if similarity_matrix is not empty
            indices = np.argsort(similarity_matrix)[0]
            for i in indices[-N:]:  # Get the last N indices ( highest similarity)
                similar_movies.append(top_movies_data[i])

        return similar_movies
    else:
        return []

def print_movie_info(movie):
    """
    Print basic movie information.

    Args:
        movie (dict): Movie data.
    """
    movie_title = movie.data.get('title', 'Unknown Title')
    movie_year = movie.data.get('year', 'Unknown Year')
    genres = movie.data.get('genres', [])
    print(f"Movie found: {movie_title} ({movie_year})")
    print(f"Genres: {genres}")

def print_similar_movies(similar_movies):
    """
    Print similar movies.

    Args:
        similar_movies (list): List of similar movies.
    """
    if not similar_movies:
        print("No similar movies found.")
        return

    print("Similar movies:")
    for movie in similar_movies:
        movie_title = movie.data.get('title', 'Unknown Title')
        movie_year = movie.data.get('year', 'Unknown Year')
        movie_genre = movie.data.get('genres', 'Unknown Genre')
        print(f"{movie_title} ({movie_year}) - {movie_genre}")

def main():
    movie_title = input("Enter a movie name: ").strip()

    # Get movie data
    movie = get_movie(movie_title)

    if movie:
        # Print movie info
        print_movie_info(movie)

        # Get similar movies
        similar_movies = get_similar_movies(movie)

        # Print similar movies
        print_similar_movies(similar_movies)
    else:
        print("No movie found.")

# Run the main function
if __name__ == "__main__":
    main()

# assosication

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset in chunks
chunksize = 1000
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize)

# Concatenate all chunks into a single DataFrame
ratings = pd.concat(chunks, ignore_index=True)

# Filter only ratings of 4.0 and above for "liked" movies
ratings_filtered = ratings[ratings['rating'] >= 4.0]

# Create a user-item matrix
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Display the rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


# assosiation model 

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset in chunks
chunksize = 1000
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize)

# Concatenate all chunks into a single DataFrame
ratings = pd.concat(chunks, ignore_index=True)

# Filter only ratings of 4.0 and above for "liked" movies
ratings_filtered = ratings[ratings['rating'] >= 4.0]

# Create a user-item matrix, where 1 represents a "liked" movie (rating 4.0 or above)
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset to map movie IDs to movie titles
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    """
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieId']
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value (descending)
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies

# Example usage:
movie_name = 'Interstellar'  # Change this to the movie title you want recommendations for
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # If the function returns an error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")


# assosiation model low resources

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.sparse import csr_matrix

# Load the dataset in chunks to avoid memory overload
chunksize = 10**6  # Increase the chunksize to process more data at once but keep it manageable
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize, usecols=['userId', 'movieId', 'rating'])

# Initialize an empty list to store filtered data
filtered_chunks = []

# Process each chunk individually
for chunk in chunks:
    # Filter ratings of 4.0 and above for "liked" movies
    filtered_chunk = chunk[chunk['rating'] >= 4.0]
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
ratings_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Create a user-item matrix in sparse format to save memory
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(pd.DataFrame(user_movie_matrix_sparse.todense(), columns=user_movie_matrix.columns), min_support=0.02, use_colnames=True)

# Generate association rules from the frequent item sets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift to find the most interesting ones
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset to map movie IDs to movie titles
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    """
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieId']
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value (descending)
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies

# Example usage:
movie_name = 'Interstellar'  # Change this to the movie title you want recommendations for
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # If the function returns an error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")


# final try 

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.sparse import csr_matrix
import re

# Function to clean movie titles by removing special characters and lowercasing
def clean_movie_title(title):
    # Remove anything in parentheses (like year)
    title = re.sub(r'\(.*?\)', '', title)
    # Remove extra spaces and lowercase the title
    title = re.sub(r'\s+', ' ', title).strip().lower()
    return title

# Load the dataset in chunks
chunksize = 10**6
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize, usecols=['userId', 'movieId', 'rating'])

# Initialize an empty list to store filtered data
filtered_chunks = []

# Process each chunk individually
for chunk in chunks:
    # Filter ratings of 4.0 and above for "liked" movies
    filtered_chunk = chunk[chunk['rating'] >= 4.0]
    filtered_chunks.append(filtered_chunk)

# Concatenate filtered chunks
ratings_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Create a user-item matrix in sparse format
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(pd.DataFrame(user_movie_matrix_sparse.todense(), columns=user_movie_matrix.columns), min_support=0.05, use_colnames=True)

# Generate association rules from frequent item sets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Apply the clean_movie_title function to standardize movie titles
movies['clean_title'] = movies['title'].apply(clean_movie_title)

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    Clean the input movie title for consistency.
    """
    cleaned_title = clean_movie_title(movie_title)
    movie_id = movies[movies['clean_title'] == cleaned_title]['movieId']
    
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies



In [ ]:
# Example usage:
movie_name = 'Heat (1995)'  # You can try different variations like 'interstellar'
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # Error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")

# cosine matrix improved

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'D:\abdo\AI\projects\recommender\needed features\imdb_data.csv')

# Convert data to a DataFrame
df = pd.DataFrame(data)

# Replace NaN values in relevant columns with an empty string
df['overview'] = df['overview'].fillna('')
df['reviews'] = df['reviews'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('')
df['genre'] = df['genre'].fillna('')

# Combine 'title', 'overview', 'cast', 'director', 'genre', and 'reviews' for similarity comparison
df['combined_features'] = df['title'] + ' ' + df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['reviews']

# Text Vectorization using TF-IDF (including bi-grams for better context)
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
tfidf_matrix = tfidf.fit_transform(df['combined_features'].fillna(''))

# Calculate cosine similarity between all movies
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Helper function to extract movie title
def extract_title(title):
    if pd.isna(title):
        return ''
    title = title.lower()
    title = re.sub(r'\s*\(\d{4}\)$', '', title)  # Remove year information
    return title

# Define the function to recommend movies
def recommend_movies(movie_title, cosine_sim=cosine_sim, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage: Recommend movies based on the title "Heat"
test = 'heat'
recommended_movies = recommend_movies(test)
print(f"Movies recommended based on '{test}':")
for movie in recommended_movies:
    print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, IMDb ID: {movie['imdbId']}")
    print(f"Director: {movie['director']}, Genre: {movie['genre']}")
    print(f"Cast: {movie['cast']}")
    print(f"Overview: {movie['overview']}")
    print(f"Reviews: {movie['reviews']}")
    print(f"Cover URL: {movie['cover_url']}")
    print("-" * 80)


# LSTM

In [ ]:
pip install tensorflow

# lstm 256 patch size adam w 

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Combine features into a single column for text processing
df['combined_features'] = df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['tag'] + ' ' + df['reviews']

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='AdamW', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='AdamW')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    # Check if the title is NaN or not a string, return an empty string in that case
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found
# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# lstm 256 batch adam

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Combine features into a single column for text processing
df['combined_features'] = df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['tag'] + ' ' + df['reviews']

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='Adam')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    # Check if the title is NaN or not a string, return an empty string in that case
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found
# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# another try 

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Feature Weighting
df['combined_features'] = (df['overview'] * 1 + ' ' +
                           df['cast'] * 3 + ' ' + 
                           df['director'] * 4 + ' ' + 
                           df['genre'] * 5 + ' ' + 
                           df['tag'] * 2 + ' ' + 
                           df['reviews'] * 1)
# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='Adam')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# lstm and attention  no weights

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Feature Weighting
df['combined_features'] = (df['overview'] * 6 + ' ' +
                           df['cast'] * 1 + ' ' + 
                           df['director'] * 4 + ' ' + 
                           df['genre'] * 5 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)


# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(128, return_sequences=True)(embedding_layer)
dropout_layer_1 = Dropout(0.2)(lstm_layer_1)
lstm_layer_2 = LSTM(64, return_sequences=True)(dropout_layer_1)

# Attention mechanism
attention_layer = Attention()([lstm_layer_2, lstm_layer_2])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(32, activation='relu')(attention_output)
dense_layer_2 = Dense(16, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=5, batch_size=256)

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Calculate cosine similarity based on 2D embeddings with Attention mechanism
cosine_sim_lstm_with_attention = cosine_similarity(movie_embeddings_2d_with_attention, movie_embeddings_2d_with_attention)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings with Attention mechanism
def recommend_movies_lstm_with_attention(movie_title, cosine_sim=cosine_sim_lstm_with_attention, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with LSTM model with Attention mechanism
test_movie = 'heat'
recommended_movies_lstm_with_attention = recommend_movies_lstm_with_attention(test_movie)

if not recommended_movies_lstm_with_attention:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm_with_attention:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


In [ ]:
test_movie = 'interstellar'
recommended_movies_lstm_with_attention = recommend_movies_lstm_with_attention(test_movie)

if not recommended_movies_lstm_with_attention:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm_with_attention:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

# eclidian distance

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from scipy.spatial.distance import euclidean

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Feature Weighting
df['combined_features'] = (df['overview'] * 7 + ' ' +
                           df['cast'] * 1 + ' ' + 
                           df['director'] * 4 + ' ' + 
                           df['genre'] * 4 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)


# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(128, return_sequences=True)(embedding_layer)
dropout_layer_1 = Dropout(0.2)(lstm_layer_1)
lstm_layer_2 = LSTM(64, return_sequences=True)(dropout_layer_1)

# Attention mechanism
attention_layer = Attention()([lstm_layer_2, lstm_layer_2])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(32, activation='relu')(attention_output)
dense_layer_2 = Dense(16, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=5, batch_size=256)

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Calculate Euclidean distance based on 2D embeddings with Attention mechanism
def calculate_euclidean_similarity(embeddings):
    num_movies = embeddings.shape[0]
    euclidean_sim = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(num_movies):
            euclidean_sim[i, j] = euclidean(embeddings[i], embeddings[j])
    return euclidean_sim

euclidean_sim_lstm_with_attention = calculate_euclidean_similarity(movie_embeddings_2d_with_attention)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings with Attention mechanism using Euclidean distance
def recommend_movies_lstm_with_euclidean(movie_title, euclidean_sim=euclidean_sim_lstm_with_attention, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(euclidean_sim[idx]))
        
        # Sort the movies based on similarity scores (ascending order for Euclidean distance)
        sim_scores = sorted(sim_scores, key=lambda x: x[1])
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with LSTM model with Attention mechanism using Euclidean distance
test_movie_heat = 'heat'
recommended_movies_lstm_with_euclidean_heat = recommend_movies_lstm_with_euclidean(test_movie_heat)

if not recommended_movies_lstm_with_euclidean_heat:
    print(f"'{test_movie_heat}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_heat}':")
    for movie in recommended_movies_lstm_with_euclidean_heat:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

test_movie_interstellar = 'interstellar'
recommended_movies_lstm_with_euclidean_interstellar = recommend_movies_lstm_with_euclidean(test_movie_interstellar)

if not recommended_movies_lstm_with_euclidean_interstellar:
    print(f"'{test_movie_interstellar}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_interstellar}':")
    for movie in recommended_movies_lstm_with_euclidean_interstellar:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# PCA

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# data cleaning
df['overview'] = df['overview'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['cast'] = df['cast'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['director'] = df['director'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['genre'] = df['genre'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['tag'] = df['tag'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['reviews'] = df['reviews'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))

# Feature Weighting
df['combined_features'] = (df['overview'] * 7 + ' ' +
                           df['cast'] * 1 + ' ' + 
                           df['director'] * 8 + ' ' + 
                           df['genre'] * 10 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)

# Tokenization and padding
max_words = 10000
max_sequence_len = 500

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(128, return_sequences=True)(embedding_layer)
dropout_layer_1 = Dropout(0.2)(lstm_layer_1)
lstm_layer_2 = LSTM(64, return_sequences=True)(dropout_layer_1)

# Attention mechanism
attention_layer = Attention()([lstm_layer_2, lstm_layer_2])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(32, activation='relu')(attention_output)
dense_layer_2 = Dense(16, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=5, batch_size=256)

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Reduce dimensionality of embeddings using PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
reduced_embeddings = pca.fit_transform(movie_embeddings_2d_with_attention)

# Calculate Euclidean distance based on reduced embeddings
def calculate_euclidean_similarity(embeddings):
    num_movies = embeddings.shape[0]
    euclidean_sim = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(num_movies):
            euclidean_sim[i, j] = euclidean(embeddings[i], embeddings[j])
    return euclidean_sim

euclidean_sim_lstm_with_attention_reduced = calculate_euclidean_similarity(reduced_embeddings)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on reduced embeddings with Euclidean distance
def recommend_movies_lstm_with_euclidean_reduced(movie_title, euclidean_sim=euclidean_sim_lstm_with_attention_reduced, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(euclidean_sim[idx]))
        
        # Sort the movies based on similarity scores (ascending order for Euclidean distance)
        sim_scores = sorted(sim_scores, key=lambda x: x[1])
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with reduced embeddings and Euclidean distance
test_movie_heat = 'heat'
recommended_movies_lstm_with_euclidean_reduced_heat = recommend_movies_lstm_with_euclidean_reduced(test_movie_heat)

if not recommended_movies_lstm_with_euclidean_reduced_heat:
    print(f"'{test_movie_heat}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_heat}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_heat:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

test_movie_interstellar = 'interstellar'
recommended_movies_lstm_with_euclidean_reduced_interstellar = recommend_movies_lstm_with_euclidean_reduced(test_movie_interstellar)

if not recommended_movies_lstm_with_euclidean_reduced_interstellar:
    print(f"'{test_movie_interstellar}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_interstellar}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_interstellar:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Clean text data
df['overview'] = df['overview'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['cast'] = df['cast'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['director'] = df['director'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['genre'] = df['genre'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['tag'] = df['tag'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['reviews'] = df['reviews'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))

# Feature Weighting
df['combined_features'] = (df['overview'] * 7 + ' ' +
                           df['cast'] * 2 + ' ' + 
                           df['director'] * 5 + ' ' + 
                           df['genre'] * 6 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(256, return_sequences=True)(embedding_layer)
dropout_layer_1 = Dropout(0.3)(lstm_layer_1)
lstm_layer_2 = LSTM(128, return_sequences=True)(dropout_layer_1)
dropout_layer_2 = Dropout(0.3)(lstm_layer_2)
lstm_layer_3 = LSTM(64, return_sequences=True)(dropout_layer_2)

# Attention mechanism
attention_layer = Attention()([lstm_layer_3, lstm_layer_3])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(64, activation='relu')(attention_output)
dense_layer_2 = Dense(32, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=5, batch_size=256)

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Reduce dimensionality of embeddings using PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
reduced_embeddings = pca.fit_transform(movie_embeddings_2d_with_attention)

# Calculate Euclidean distance based on reduced embeddings
def calculate_euclidean_similarity(embeddings):
    num_movies = embeddings.shape[0]
    euclidean_sim = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(num_movies):
            euclidean_sim[i, j] = euclidean(embeddings[i], embeddings[j])
    return euclidean_sim

euclidean_sim_lstm_with_attention_reduced = calculate_euclidean_similarity(reduced_embeddings)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on reduced embeddings with Euclidean distance
def recommend_movies_lstm_with_euclidean_reduced(movie_title, euclidean_sim=euclidean_sim_lstm_with_attention_reduced, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(euclidean_sim[idx]))
        
        # Sort the movies based on similarity scores (ascending order for Euclidean distance)
        sim_scores = sorted(sim_scores, key=lambda x: x[1])
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with reduced embeddings and Euclidean distance
test_movie_heat = 'heat'
recommended_movies_lstm_with_euclidean_reduced_heat = recommend_movies_lstm_with_euclidean_reduced(test_movie_heat)

if not recommended_movies_lstm_with_euclidean_reduced_heat:
    print(f"'{test_movie_heat}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_heat}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_heat:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")


In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Clean text data
df['overview'] = df['overview'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['cast'] = df['cast'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['director'] = df['director'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['genre'] = df['genre'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['tag'] = df['tag'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['reviews'] = df['reviews'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))

# Feature Weighting
df['combined_features'] = (df['overview'] * 7 + ' ' +
                           df['cast'] * 2 + ' ' + 
                           df['director'] * 5 + ' ' + 
                           df['genre'] * 6 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(256, return_sequences=True, kernel_regularizer=l2(0.01))(embedding_layer)
dropout_layer_1 = Dropout(0.5)(lstm_layer_1)
lstm_layer_2 = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(lstm_layer_2)
lstm_layer_3 = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01))(dropout_layer_2)

# Attention mechanism
attention_layer = Attention()([lstm_layer_3, lstm_layer_3])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(attention_output)
dense_layer_2 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=20, batch_size=256, validation_split=0.2, callbacks=[early_stopping])

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Reduce dimensionality of embeddings using PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
reduced_embeddings = pca.fit_transform(movie_embeddings_2d_with_attention)

# Calculate Euclidean distance based on reduced embeddings
def calculate_euclidean_similarity(embeddings):
    num_movies = embeddings.shape[0]
    euclidean_sim = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(num_movies):
            euclidean_sim[i, j] = euclidean(embeddings[i], embeddings[j])
    return euclidean_sim

euclidean_sim_lstm_with_attention_reduced = calculate_euclidean_similarity(reduced_embeddings)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on reduced embeddings with Euclidean distance
def recommend_movies_lstm_with_euclidean_reduced(movie_title, euclidean_sim=euclidean_sim_lstm_with_attention_reduced, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(euclidean_sim[idx]))
        
        # Sort the movies based on similarity scores (ascending order for Euclidean distance)
        sim_scores = sorted(sim_scores, key=lambda x: x[1])
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with reduced embeddings and Euclidean distance
test_movie_heat = 'heat'
recommended_movies_lstm_with_euclidean_reduced_heat = recommend_movies_lstm_with_euclidean_reduced(test_movie_heat)

if not recommended_movies_lstm_with_euclidean_reduced_heat:
    print(f"'{test_movie_heat}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_heat}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_heat:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

test_movie_interstellar = 'interstellar'
recommended_movies_lstm_with_euclidean_reduced_interstellar = recommend_movies_lstm_with_euclidean_reduced(test_movie_interstellar)

if not recommended_movies_lstm_with_euclidean_reduced_interstellar:
    print(f"'{test_movie_interstellar}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_interstellar}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_interstellar:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


In [1]:
pip install tmdbv3api pandas scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd

# Initialize TMDb with your API key
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Initialize the Movie class
movie = Movie()

def get_movie_data(movie_id):
    return movie.details(movie_id)

def get_popular_movies():
    return movie.popular()

def get_movie_list():
    popular_movies = get_popular_movies()
    movies_data = []
    for m in popular_movies['results']:
        movie_id = m['id']
        details = get_movie_data(movie_id)
        movies_data.append({
            'id': movie_id,
            'title': details.get('title'),
            'genres': ', '.join([g['name'] for g in details.get('genres', [])]),
            'rating': details.get('vote_average'),
            'overview': details.get('overview'),
        })
    return pd.DataFrame(movies_data)

# Fetch movie list
movies_df = get_movie_list()
print(movies_df.head())


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def recommend_movies(title, movies_df):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['overview'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 recommendations
    movie_indices = [i[0] for i in sim_scores]
    return movies_df['title'].iloc[movie_indices]

# Example usage
recommended_movies = recommend_movies('The Shawshank Redemption', movies_df)
print(recommended_movies)


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Initialize TMDb with your API key
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Initialize the Movie class
movie = Movie()

def get_movie_data(movie_id):
    """Fetch detailed movie information from TMDb."""
    return movie.details(movie_id)

def get_popular_movies():
    """Fetch popular movies from TMDb."""
    return movie.popular()

def get_movie_list():
    """Fetch and return a DataFrame of popular movies with their details."""
    popular_movies = get_popular_movies()
    movies_data = []
    for m in popular_movies['results']:
        movie_id = m['id']
        details = get_movie_data(movie_id)
        movies_data.append({
            'id': movie_id,
            'title': details.get('title'),
            'genres': ', '.join([g['name'] for g in details.get('genres', [])]),
            'rating': details.get('vote_average'),
            'overview': details.get('overview'),
        })
    return pd.DataFrame(movies_data)

def recommend_movies(title, movies_df):
    """Recommend movies based on the given title using a content-based filtering approach."""
    if 'title' not in movies_df.columns:
        raise ValueError("The 'title' column is missing from the DataFrame")
    
    if title.lower() not in movies_df['title'].str.lower().values:
        raise ValueError(f"The title '{title}' is not in the DataFrame")
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['overview'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    indices = pd.Series(movies_df.index, index=movies_df['title'].str.lower()).drop_duplicates()
    
    if title.lower() not in indices:
        raise ValueError(f"The title '{title}' is not found in the index")
    
    idx = indices[title.lower()]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 recommendations
    movie_indices = [i[0] for i in sim_scores]
    
    return movies_df['title'].iloc[movie_indices]

# Main Workflow
if __name__ == "__main__":
    # Fetch movie list
    movies_df = get_movie_list()
    
    # Print titles to verify
    print("Available movie titles:")
    print(movies_df['title'].head())

    # Example recommendation
    try:
        recommended_movies = recommend_movies('The Shawshank Redemption', movies_df)
        print("\nRecommended Movies:")
        print(recommended_movies)
    except ValueError as e:
        print(e)


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Initialize TMDb with your API key
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Initialize the Movie class
movie = Movie()

def get_movie_data(movie_id):
    """Fetch detailed movie information from TMDb."""
    return movie.details(movie_id)

def get_popular_movies():
    """Fetch popular movies from TMDb."""
    return movie.popular()

def search_movies(query):
    """Search for movies by title using TMDb."""
    search = tmdb.Search()
    results = search.movie(query=query)
    movies_data = []
    for result in results['results']:
        movie_id = result['id']
        details = get_movie_data(movie_id)
        movies_data.append({
            'id': movie_id,
            'title': details.get('title'),
            'genres': ', '.join([g['name'] for g in details.get('genres', [])]),
            'rating': details.get('vote_average'),
            'overview': details.get('overview'),
        })
    return pd.DataFrame(movies_data)

def recommend_movies(title, movies_df):
    """Recommend movies based on the given title using a content-based filtering approach."""
    if 'title' not in movies_df.columns:
        raise ValueError("The 'title' column is missing from the DataFrame")
    
    if title.lower() not in movies_df['title'].str.lower().values:
        raise ValueError(f"The title '{title}' is not in the DataFrame")
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['overview'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    indices = pd.Series(movies_df.index, index=movies_df['title'].str.lower()).drop_duplicates()
    
    if title.lower() not in indices:
        raise ValueError(f"The title '{title}' is not found in the index")
    
    idx = indices[title.lower()]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 recommendations
    movie_indices = [i[0] for i in sim_scores]
    
    return movies_df['title'].iloc[movie_indices]

# Main Workflow
if __name__ == "__main__":
    # Fetch movie list
    movies_df = get_movie_list()
    
    # Print titles to verify
    print("Available movie titles:")
    print(movies_df['title'].head())

    # Search for a specific movie
    searched_movies_df = search_movies('The Shawshank Redemption')
    print("\nSearched Movies:")
    print(searched_movies_df)

    # Example recommendation
    try:
        recommended_movies = recommend_movies('The Shawshank Redemption', searched_movies_df)
        print("\nRecommended Movies:")
        print(recommended_movies)
    except ValueError as e:
        print(e)


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import requests

# Initialize TMDb with your API key
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Initialize the Movie class
movie = Movie()

def fetch_genre_names():
    """Fetch the genre names from TMDb and return a dictionary."""
    url = "https://api.themoviedb.org/3/genre/movie/list"
    params = {"api_key": tmdb.api_key, "language": "en-US"}
    response = requests.get(url, params=params)
    data = response.json()
    return {g['id']: g['name'] for g in data['genres']}

def get_movie_data(movie_id):
    """Fetch detailed movie information from TMDb."""
    return movie.details(movie_id)

def get_popular_movies():
    """Fetch popular movies from TMDb and convert to DataFrame."""
    popular_movies = movie.popular()
    genre_names = fetch_genre_names()
    
    movies_data = []
    for result in popular_movies['results']:
        genres = [genre_names.get(g_id, 'Unknown') for g_id in result.get('genre_ids', [])]
        movies_data.append({
            'id': result['id'],
            'title': result.get('title'),
            'genres': ', '.join(genres),
            'rating': result.get('vote_average'),
            'overview': result.get('overview'),
        })
    return pd.DataFrame(movies_data)

def search_movies(query):
    """Search for movies by title using TMDb."""
    url = "https://api.themoviedb.org/3/search/movie"
    params = {"api_key": tmdb.api_key, "query": query, "language": "en-US"}
    response = requests.get(url, params=params)
    search_results = response.json()
    genre_names = fetch_genre_names()
    
    movies_data = []
    for result in search_results['results']:
        movie_id = result['id']
        details = get_movie_data(movie_id)
        genres = [genre_names.get(g_id, 'Unknown') for g_id in details.get('genres', [])]
        movies_data.append({
            'id': movie_id,
            'title': details.get('title'),
            'genres': ', '.join(genres),
            'rating': details.get('vote_average'),
            'overview': details.get('overview'),
        })
    return pd.DataFrame(movies_data)

def recommend_movies(title, movies_df):
    """Recommend movies based on the given title using a content-based filtering approach."""
    if 'title' not in movies_df.columns:
        raise ValueError("The 'title' column is missing from the DataFrame")
    
    if title.lower() not in movies_df['title'].str.lower().values:
        raise ValueError(f"The title '{title}' is not in the DataFrame")
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['overview'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    indices = pd.Series(movies_df.index, index=movies_df['title'].str.lower()).drop_duplicates()
    
    if title.lower() not in indices:
        raise ValueError(f"The title '{title}' is not found in the index")
    
    idx = indices[title.lower()]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 recommendations
    movie_indices = [i[0] for i in sim_scores]
    
    return movies_df['title'].iloc[movie_indices]

# Main Workflow
if __name__ == "__main__":
    # Fetch movie list
    movies_df = get_popular_movies()
    
    # Print titles to verify
    print("Available movie titles:")
    print(movies_df['title'].head())

    # Search for a specific movie
    searched_movies_df = search_movies('The Shawshank Redemption')
    print("\nSearched Movies:")
    print(searched_movies_df)

    # Example recommendation
    try:
        recommended_movies = recommend_movies('The Shawshank Redemption', searched_movies_df)
        print("\nRecommended Movies:")
        print(recommended_movies)
    except ValueError as e:
        print(e)


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import requests

# Initialize TMDb with your API key
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Initialize the Movie class
movie = Movie()

# Fetch genre names once
def fetch_genre_names():
    """Fetch the genre names from TMDb and return a dictionary."""
    url = "https://api.themoviedb.org/3/genre/movie/list"
    params = {"api_key": tmdb.api_key, "language": "en-US"}
    response = requests.get(url, params=params)
    data = response.json()
    return {g['id']: g['name'] for g in data['genres']}

# Fetch movie data
def get_movie_data(movie_id):
    """Fetch detailed movie information from TMDb."""
    url = f"https://api.themoviedb.org/3/movie/{movie_id}"
    params = {"api_key": tmdb.api_key, "language": "en-US"}
    response = requests.get(url, params=params)
    return response.json()

def get_popular_movies():
    """Fetch popular movies from TMDb and convert to DataFrame."""
    popular_movies = movie.popular()
    genre_names = fetch_genre_names()
    
    movies_data = []
    for result in popular_movies['results']:
        genres = [genre_names.get(g_id, 'Unknown') for g_id in result.get('genre_ids', [])]
        movies_data.append({
            'id': result['id'],
            'title': result.get('title'),
            'genres': ', '.join(genres),
            'rating': result.get('vote_average'),
            'overview': result.get('overview'),
        })
    return pd.DataFrame(movies_data)

def search_movies(query):
    """Search for movies by title using TMDb."""
    url = "https://api.themoviedb.org/3/search/movie"
    params = {"api_key": tmdb.api_key, "query": query, "language": "en-US"}
    response = requests.get(url, params=params)
    search_results = response.json()
    genre_names = fetch_genre_names()
    
    movies_data = []
    for result in search_results['results']:
        movie_id = result['id']
        details = get_movie_data(movie_id)
        # Ensure genres is a list of dictionaries
        genres = [genre_names.get(g['id'], 'Unknown') for g in details.get('genres', [])]
        movies_data.append({
            'id': movie_id,
            'title': details.get('title'),
            'genres': ', '.join(genres),
            'rating': details.get('vote_average'),
            'overview': details.get('overview'),
        })
    return pd.DataFrame(movies_data)

def recommend_movies(title, movies_df):
    """Recommend movies based on the given title using a content-based filtering approach."""
    if 'title' not in movies_df.columns:
        raise ValueError("The 'title' column is missing from the DataFrame")
    
    if title.lower() not in movies_df['title'].str.lower().values:
        raise ValueError(f"The title '{title}' is not in the DataFrame")
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['overview'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    indices = pd.Series(movies_df.index, index=movies_df['title'].str.lower()).drop_duplicates()
    
    if title.lower() not in indices:
        raise ValueError(f"The title '{title}' is not found in the index")
    
    idx = indices[title.lower()]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 recommendations
    movie_indices = [i[0] for i in sim_scores]
    
    return movies_df.iloc[movie_indices]['title']

# Main Workflow
if __name__ == "__main__":
    # Fetch movie list
    movies_df = get_popular_movies()
    
    # Print titles to verify
    print("Available movie titles:")
    print(movies_df['title'].head())

    # Search for a specific movie
    searched_movies_df = search_movies('The Shawshank Redemption')
    print("\nSearched Movies:")
    print(searched_movies_df)

    # Example recommendation
    try:
        recommended_movies = recommend_movies('The Shawshank Redemption', searched_movies_df)
        print("\nRecommended Movies:")
        print(recommended_movies)
    except ValueError as e:
        print(e)


In [ ]:
from imdb import IMDb

# Initialize IMDbPY instance
ia = IMDb()

# Function to search for a movie
def get_movie_by_name(movie_name):
    movies = ia.search_movie(movie_name)
    if not movies:
        return None
    return movies[0]

# Function to fetch movies by genre and year
def get_movies_by_genre_and_year(genres, year, keywords):
    all_movies = []
    for genre in genres:
        try:
            # Search for movies by genre keyword
            movies = ia.search_movie(genre)
            # Filter by year (within +/- 10 years)
            filtered_movies = [m for m in movies if abs(m.get('year', 0) - year) <= 10]
            all_movies.extend(filtered_movies)
        except Exception as e:
            print(f"Error fetching movies for genre '{genre}': {e}")
    return all_movies

# Function to get user preferences for weights
def get_user_preferences():
    print("Assign weights to the following features (0-5):")
    genre_weight = int(input("Genres (0-5): "))
    director_weight = int(input("Directors (0-5): "))
    actor_weight = int(input("Actors (0-5): "))
    keyword_weight = int(input("Keywords (0-5): "))
    rating_weight = int(input("Ratings (0-5): "))
    
    return {
        'genre_weight': genre_weight,
        'director_weight': director_weight,
        'actor_weight': actor_weight,
        'keyword_weight': keyword_weight,
        'rating_weight': rating_weight
    }

# Function to get similar movies with user-specified weights
def recommend_similar_movies(movie, user_weights, num_recommendations=3):
    ia.update(movie)
    
    # Extract features to compare
    movie_genres = set(movie.get('genres', []))
    movie_directors = set([director['name'] for director in movie.get('directors', [])])
    movie_actors = set([actor['name'] for actor in movie.get('cast', [])[:5]])
    movie_keywords = set(movie.get('keywords', []))
    movie_rating = movie.get('rating', 0)
    movie_year = movie.get('year', 0)
    
    # Fetch a larger dataset by genre, year, and keywords
    similar_movies = get_movies_by_genre_and_year(movie_genres, movie_year, movie_keywords)
    
    recommendations = []
    
    for other_movie in similar_movies:
        ia.update(other_movie)
        
        # Compare genres (user-defined weight)
        other_genres = set(other_movie.get('genres', []))
        genre_similarity = len(movie_genres & other_genres) * user_weights['genre_weight']
        
        # Compare directors (user-defined weight)
        other_directors = set([director['name'] for director in other_movie.get('directors', [])])
        director_similarity = len(movie_directors & other_directors) * user_weights['director_weight']
        
        # Compare actors (user-defined weight)
        other_actors = set([actor['name'] for actor in other_movie.get('cast', [])[:5]])
        actor_similarity = len(movie_actors & other_actors) * user_weights['actor_weight']
        
        # Compare keywords (user-defined weight)
        other_keywords = set(other_movie.get('keywords', []))
        keyword_similarity = len(movie_keywords & other_keywords) * user_weights['keyword_weight']
        
        # Compare IMDb ratings (user-defined weight)
        other_rating = other_movie.get('rating', 0)
        rating_difference = 1 - abs(movie_rating - other_rating) / 10  # Normalized difference
        rating_similarity = rating_difference * user_weights['rating_weight']
        
        # Total score based on user-specified weights
        score = genre_similarity + director_similarity + actor_similarity + keyword_similarity + rating_similarity
        
        if score > 0:
            recommendations.append((other_movie, score))
    
    # Sort by score and return top recommendations
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return [movie[0] for movie in recommendations[:num_recommendations]]

# Main function
def recommend_movies(movie_name):
    # Get user preferences for weights
    user_weights = get_user_preferences()
    
    # Get the user's movie
    movie = get_movie_by_name(movie_name)
    
    if not movie:
        return "Movie not found. Please try a different name."
    
    print(f"Movie found: {movie['title']} ({movie['year']})")
    
    # Get 3 recommended movies based on user preferences
    recommendations = recommend_similar_movies(movie, user_weights)
    
    # Output the recommendations
    if recommendations:
        print("\nRecommended movies:")
        for rec in recommendations:
            print(f"- {rec['title']} ({rec['year']}) | Rating: {rec.get('rating', 'N/A')}")
    else:
        print("No similar movies found.")

# Example usage
movie_name = input("Enter a movie name: ")
recommend_movies(movie_name)


In [ ]:
from imdb import IMDb

# Initialize IMDbPY instance
ia = IMDb()

# Function to search for a movie
def get_movie_by_name(movie_name):
    movies = ia.search_movie(movie_name)
    if not movies:
        return None
    return movies[0]

# Function to fetch movies by genre and year
def get_movies_by_genre_and_year(genres, year, keywords, num_results=100):
    all_movies = []
    for genre in genres:
        try:
            # Search for movies by genre keyword
            movies = ia.search_movie(genre)
            # Filter results to movies only and by year range
            filtered_movies = [m for m in movies if 'movie' in m['kind'] and abs(m.get('year', 0) - year) <= 10]
            all_movies.extend(filtered_movies)
        except Exception as e:
            print(f"Error fetching movies for genre '{genre}': {e}")
    # Remove duplicates and limit to num_results
    all_movies = list({m['title']: m for m in all_movies}.values())
    return all_movies[:num_results]

# Function to get user preferences for weights
def get_user_preferences():
    print("Assign weights to the following features (0-5):")
    genre_weight = int(input("Genres (0-5): "))
    director_weight = int(input("Directors (0-5): "))
    actor_weight = int(input("Actors (0-5): "))
    keyword_weight = int(input("Keywords (0-5): "))
    rating_weight = int(input("Ratings (0-5): "))
    
    return {
        'genre_weight': genre_weight,
        'director_weight': director_weight,
        'actor_weight': actor_weight,
        'keyword_weight': keyword_weight,
        'rating_weight': rating_weight
    }

# Function to get similar movies with user-specified weights
def recommend_similar_movies(movie, user_weights, num_recommendations=3):
    ia.update(movie)
    
    # Extract features to compare
    movie_genres = set(movie.get('genres', []))
    movie_directors = set([director['name'] for director in movie.get('directors', [])])
    movie_actors = set([actor['name'] for actor in movie.get('cast', [])[:5]])
    movie_keywords = set(movie.get('keywords', []))
    movie_rating = movie.get('rating', 0)
    movie_year = movie.get('year', 0)
    
    # Fetch a larger dataset by genre, year, and keywords
    similar_movies = get_movies_by_genre_and_year(movie_genres, movie_year, movie_keywords)
    
    recommendations = []
    
    for other_movie in similar_movies:
        ia.update(other_movie)
        
        # Filter out TV shows
        if 'movie' not in other_movie.get('kind', ''):
            continue
        
        # Compare genres (user-defined weight)
        other_genres = set(other_movie.get('genres', []))
        genre_similarity = len(movie_genres & other_genres) * user_weights['genre_weight']
        
        # Compare directors (user-defined weight)
        other_directors = set([director['name'] for director in other_movie.get('directors', [])])
        director_similarity = len(movie_directors & other_directors) * user_weights['director_weight']
        
        # Compare actors (user-defined weight)
        other_actors = set([actor['name'] for actor in other_movie.get('cast', [])[:5]])
        actor_similarity = len(movie_actors & other_actors) * user_weights['actor_weight']
        
        # Compare keywords (user-defined weight)
        other_keywords = set(other_movie.get('keywords', []))
        keyword_similarity = len(movie_keywords & other_keywords) * user_weights['keyword_weight']
        
        # Compare IMDb ratings (user-defined weight)
        other_rating = other_movie.get('rating', 0)
        rating_difference = 1 - abs(movie_rating - other_rating) / 10  # Normalized difference
        rating_similarity = rating_difference * user_weights['rating_weight']
        
        # Total score based on user-specified weights
        score = genre_similarity + director_similarity + actor_similarity + keyword_similarity + rating_similarity
        
        if score > 0:
            recommendations.append((other_movie, score))
    
    # Sort by score and return top recommendations
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return [movie[0] for movie in recommendations[:num_recommendations]]

# Main function
def recommend_movies(movie_name):
    # Get user preferences for weights
    user_weights = get_user_preferences()
    
    # Get the user's movie
    movie = get_movie_by_name(movie_name)
    
    if not movie:
        return "Movie not found. Please try a different name."
    
    print(f"Movie found: {movie['title']} ({movie['year']})")
    
    # Get 3 recommended movies based on user preferences
    recommendations = recommend_similar_movies(movie, user_weights)
    
    # Output the recommendations
    if recommendations:
        print("\nRecommended movies:")
        for rec in recommendations:
            print(f"- {rec['title']} ({rec['year']}) | Rating: {rec.get('rating', 'N/A')}")
    else:
        print("No similar movies found.")

# Example usage
movie_name = input("Enter a movie name: ")
recommend_movies(movie_name)


In [ ]:
from imdb import IMDb

# Initialize IMDbPY instance
ia = IMDb()

# Function to search for a movie
def get_movie_by_name(movie_name):
    movies = ia.search_movie(movie_name)
    if not movies:
        return None
    return movies[0]

# Function to fetch movies by genre and year with additional attributes
def get_movies_by_genre_and_year(genres, year, keywords, num_results=100):
    all_movies = []
    for genre in genres:
        try:
            # Search for movies by genre keyword
            movies = ia.search_movie(genre)
            # Filter results to movies only and by year range
            filtered_movies = [m for m in movies if 'movie' in m['kind'] and abs(m.get('year', 0) - year) <= 10]
            all_movies.extend(filtered_movies)
        except Exception as e:
            print(f"Error fetching movies for genre '{genre}': {e}")
    # Remove duplicates and limit to num_results
    all_movies = list({m['title']: m for m in all_movies}.values())
    return all_movies[:num_results]

# Function to get user preferences for weights
def get_user_preferences():
    print("Assign weights to the following features (0-5):")
    genre_weight = int(input("Genres (0-5): "))
    director_weight = int(input("Directors (0-5): "))
    actor_weight = int(input("Actors (0-5): "))
    keyword_weight = int(input("Keywords (0-5): "))
    rating_weight = int(input("Ratings (0-5): "))
    overview_weight = int(input("Overview (0-5): "))
    
    return {
        'genre_weight': genre_weight,
        'director_weight': director_weight,
        'actor_weight': actor_weight,
        'keyword_weight': keyword_weight,
        'rating_weight': rating_weight,
        'overview_weight': overview_weight
    }

# Function to calculate similarity based on features
def calculate_similarity(movie, other_movie, user_weights):
    # Extract features to compare
    movie_genres = set(movie.get('genres', []))
    movie_directors = set([director['name'] for director in movie.get('directors', [])])
    movie_actors = set([actor['name'] for actor in movie.get('cast', [])[:5]])
    movie_keywords = set(movie.get('keywords', []))
    movie_rating = movie.get('rating', 0)
    movie_overview = movie.get('plot', [''])[0]  # Overview (plot)
    
    other_genres = set(other_movie.get('genres', []))
    other_directors = set([director['name'] for director in other_movie.get('directors', [])])
    other_actors = set([actor['name'] for actor in other_movie.get('cast', [])[:5]])
    other_keywords = set(other_movie.get('keywords', []))
    other_rating = other_movie.get('rating', 0)
    other_overview = other_movie.get('plot', [''])[0]  # Overview (plot)
    
    # Calculate similarity
    genre_similarity = len(movie_genres & other_genres) * user_weights['genre_weight']
    director_similarity = len(movie_directors & other_directors) * user_weights['director_weight']
    actor_similarity = len(movie_actors & other_actors) * user_weights['actor_weight']
    keyword_similarity = len(movie_keywords & other_keywords) * user_weights['keyword_weight']
    rating_similarity = (1 - abs(movie_rating - other_rating) / 10) * user_weights['rating_weight']
    
    # Simple overview similarity based on common words
    common_words = len(set(movie_overview.split()) & set(other_overview.split()))
    overview_similarity = (common_words / max(len(movie_overview.split()), len(other_overview.split()))) * user_weights['overview_weight']
    
    # Total score based on user-specified weights
    return genre_similarity + director_similarity + actor_similarity + keyword_similarity + rating_similarity + overview_similarity

# Function to get similar movies with user-specified weights
def recommend_similar_movies(movie, user_weights, num_recommendations=3):
    ia.update(movie)
    
    # Extract features of the input movie
    movie_genres = set(movie.get('genres', []))
    movie_year = movie.get('year', 0)
    movie_keywords = set(movie.get('keywords', []))
    
    # Fetch a larger dataset by genre, year, and keywords
    similar_movies = get_movies_by_genre_and_year(movie_genres, movie_year, movie_keywords)
    
    recommendations = []
    
    for other_movie in similar_movies:
        ia.update(other_movie)
        
        # Filter out TV shows
        if 'movie' not in other_movie.get('kind', ''):
            continue
        
        # Calculate similarity score
        score = calculate_similarity(movie, other_movie, user_weights)
        
        if score > 0:
            recommendations.append((other_movie, score))
    
    # Sort by score and return top recommendations
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return [movie[0] for movie in recommendations[:num_recommendations]]

# Main function
def recommend_movies(movie_name):
    # Get user preferences for weights
    user_weights = get_user_preferences()
    
    # Get the user's movie
    movie = get_movie_by_name(movie_name)
    
    if not movie:
        return "Movie not found. Please try a different name."
    
    print(f"Movie found: {movie['title']} ({movie['year']})")
    
    # Get 3 recommended movies based on user preferences
    recommendations = recommend_similar_movies(movie, user_weights)
    
    # Output the recommendations
    if recommendations:
        print("\nRecommended movies:")
        for rec in recommendations:
            print(f"- {rec['title']} ({rec['year']}) | Rating: {rec.get('rating', 'N/A')}")
    else:
        print("No similar movies found.")

# Example usage
movie_name = input("Enter a movie name: ")
recommend_movies(movie_name)


In [ ]:
from imdb import IMDb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize IMDbPY instance
ia = IMDb()

# Function to get movie features
def get_movie_features(movie):
    ia.update(movie)
    return {
        'title': movie.get('title', ''),
        'year': movie.get('year', ''),
        'genres': movie.get('genres', []),
        'directors': [director['name'] for director in movie.get('directors', [])],
        'actors': [actor['name'] for actor in movie.get('cast', [])[:5]],
        'keywords': movie.get('keywords', []),
        'rating': movie.get('rating', 0),
        'overview': movie.get('plot', [''])[0]  # Overview (plot)
    }

# Function to get user preferences for weights
def get_user_preferences():
    print("Assign weights to the following features (0-5):")
    genre_weight = int(input("Genres (0-5): "))
    director_weight = int(input("Directors (0-5): "))
    actor_weight = int(input("Actors (0-5): "))
    keyword_weight = int(input("Keywords (0-5): "))
    rating_weight = int(input("Ratings (0-5): "))
    overview_weight = int(input("Overview (0-5): "))
    
    return {
        'genre_weight': genre_weight,
        'director_weight': director_weight,
        'actor_weight': actor_weight,
        'keyword_weight': keyword_weight,
        'rating_weight': rating_weight,
        'overview_weight': overview_weight
    }

# Function to calculate similarity between two movies
def calculate_similarity(movie, other_movie, user_weights):
    movie_features = get_movie_features(movie)
    other_features = get_movie_features(other_movie)
    
    # Similarity based on genres
    genre_similarity = len(set(movie_features['genres']) & set(other_features['genres'])) * user_weights['genre_weight']
    
    # Similarity based on directors
    director_similarity = len(set(movie_features['directors']) & set(other_features['directors'])) * user_weights['director_weight']
    
    # Similarity based on actors
    actor_similarity = len(set(movie_features['actors']) & set(other_features['actors'])) * user_weights['actor_weight']
    
    # Similarity based on keywords
    keyword_similarity = len(set(movie_features['keywords']) & set(other_features['keywords'])) * user_weights['keyword_weight']
    
    # Similarity based on IMDb ratings
    rating_similarity = (1 - abs(movie_features['rating'] - other_features['rating']) / 10) * user_weights['rating_weight']
    
    # Similarity based on overviews using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([movie_features['overview'], other_features['overview']])
    overview_similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0] * user_weights['overview_weight']
    
    # Total score
    return genre_similarity + director_similarity + actor_similarity + keyword_similarity + rating_similarity + overview_similarity

# Function to recommend similar movies
def recommend_similar_movies(movie, user_weights, num_recommendations=3):
    ia.update(movie)
    
    movie_features = get_movie_features(movie)
    genres = movie_features['genres']
    year = movie_features['year']
    keywords = movie_features['keywords']
    
    # Fetch a large dataset
    similar_movies = get_movies_by_genre_and_year(genres, year, keywords)
    
    recommendations = []
    
    for other_movie in similar_movies:
        ia.update(other_movie)
        
        # Filter out TV shows
        if 'movie' not in other_movie.get('kind', ''):
            continue
        
        # Calculate similarity score
        score = calculate_similarity(movie, other_movie, user_weights)
        
        if score > 0:
            recommendations.append((other_movie, score))
    
    # Sort by score and return top recommendations
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return [movie[0] for movie in recommendations[:num_recommendations]]

# Main function
def recommend_movies(movie_name):
    user_weights = get_user_preferences()
    movie = get_movie_by_name(movie_name)
    
    if not movie:
        return "Movie not found. Please try a different name."
    
    print(f"Movie found: {movie['title']} ({movie['year']})")
    
    recommendations = recommend_similar_movies(movie, user_weights)
    
    if recommendations:
        print("\nRecommended movies:")
        for rec in recommendations:
            print(f"- {rec['title']} ({rec['year']}) | Rating: {rec.get('rating', 'N/A')}")
    else:
        print("No similar movies found.")

# Example usage
movie_name = input("Enter a movie name: ")
recommend_movies(movie_name)


In [3]:
import nltk
from imdb import IMDb

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize IMDb instance and NLTK components
ia = IMDb()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def get_movie_features(movie):
    ia.update(movie)
    return {
        'title': movie.get('title', ''),
        'year': movie.get('year', ''),
        'genres': movie.get('genres', []),
        'directors': [director['name'] for director in movie.get('directors', [])],
        'actors': [actor['name'] for actor in movie.get('cast', [])[:5]],
        'keywords': movie.get('keywords', []),
        'rating': movie.get('rating', 0),
        'overview': movie.get('plot', [''])[0],
        'release_date': movie.get('release_date', ''),
        'votes': movie.get('votes', 0),
        'runtime': movie.get('runtime', 0),
        'language': movie.get('languages', []),
        'country': movie.get('countries', []),
        'certification': movie.get('certificates', []),
        'soundtrack': movie.get('soundtracks', []),
        'production_companies': [company['name'] for company in movie.get('production companies', [])],
        'alternate_titles': movie.get('aka', []),
        'tagline': movie.get('tagline', ''),
        'box_office': movie.get('box office', {})
    }

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

def calculate_text_similarity(text1, text2):
    # Placeholder for text similarity implementation
    return 1 if text1.lower() == text2.lower() else 0

def calculate_date_similarity(date1, date2):
    return 1 if date1[:4] == date2[:4] else 0

def calculate_box_office_similarity(box_office1, box_office2):
    return 1 - abs(box_office1.get('gross', 0) - box_office2.get('gross', 0)) / max(box_office1.get('gross', 1), box_office2.get('gross', 1))

def calculate_similarity(movie, other_movie, user_weights):
    movie_features = get_movie_features(movie)
    other_features = get_movie_features(other_movie)
    
    genre_similarity = len(set(movie_features['genres']) & set(other_features['genres'])) * user_weights['genre_weight']
    director_similarity = len(set(movie_features['directors']) & set(other_features['directors'])) * user_weights['director_weight']
    actor_similarity = len(set(movie_features['actors']) & set(other_features['actors'])) * user_weights['actor_weight']
    keyword_similarity = len(set(movie_features['keywords']) & set(other_features['keywords'])) * user_weights['keyword_weight']
    rating_similarity = (1 - abs(movie_features['rating'] - other_features['rating']) / 10) * user_weights['rating_weight']
    overview_similarity = calculate_text_similarity(preprocess_text(movie_features['overview']), preprocess_text(other_features['overview'])) * user_weights['overview_weight']
    release_date_similarity = calculate_date_similarity(movie_features['release_date'], other_features['release_date']) * user_weights['release_date_weight']
    votes_similarity = (1 - abs(movie_features['votes'] - other_features['votes']) / max(movie_features['votes'], 1)) * user_weights['votes_weight']
    runtime_similarity = (1 - abs(movie_features['runtime'] - other_features['runtime']) / max(movie_features['runtime'], 1)) * user_weights['runtime_weight']
    language_similarity = len(set(movie_features['language']) & set(other_features['language'])) * user_weights['language_weight']
    country_similarity = len(set(movie_features['country']) & set(other_features['country'])) * user_weights['country_weight']
    certification_similarity = len(set(movie_features['certification']) & set(other_features['certification'])) * user_weights['certification_weight']
    soundtrack_similarity = len(set(movie_features['soundtrack']) & set(other_features['soundtrack'])) * user_weights['soundtrack_weight']
    production_companies_similarity = len(set(movie_features['production_companies']) & set(other_features['production_companies'])) * user_weights['production_companies_weight']
    alternate_titles_similarity = len(set(movie_features['alternate_titles']) & set(other_features['alternate_titles'])) * user_weights['alternate_titles_weight']
    tagline_similarity = calculate_text_similarity(movie_features['tagline'], other_features['tagline']) * user_weights['tagline_weight']
    box_office_similarity = calculate_box_office_similarity(movie_features['box_office'], other_features['box_office']) * user_weights['box_office_weight']
    
    return (genre_similarity + director_similarity + actor_similarity + keyword_similarity + rating_similarity +
            overview_similarity + release_date_similarity + votes_similarity + runtime_similarity +
            language_similarity + country_similarity + certification_similarity + soundtrack_similarity +
            production_companies_similarity + alternate_titles_similarity + tagline_similarity +
            box_office_similarity)

def get_user_preferences():
    print("Assign weights to the following features (0-5):")
    genre_weight = int(input("Genres (0-5): "))
    director_weight = int(input("Directors (0-5): "))
    actor_weight = int(input("Actors (0-5): "))
    keyword_weight = int(input("Keywords (0-5): "))
    rating_weight = int(input("Ratings (0-5): "))
    overview_weight = int(input("Overview (0-5): "))
    release_date_weight = int(input("Release Date (0-5): "))
    votes_weight = int(input("Votes (0-5): "))
    runtime_weight = int(input("Runtime (0-5): "))
    language_weight = int(input("Language (0-5): "))
    country_weight = int(input("Country (0-5): "))
    certification_weight = int(input("Certification (0-5): "))
    soundtrack_weight = int(input("Soundtrack (0-5): "))
    production_companies_weight = int(input("Production Companies (0-5): "))
    alternate_titles_weight = int(input("Alternate Titles (0-5): "))
    tagline_weight = int(input("Tagline (0-5): "))
    box_office_weight = int(input("Box Office (0-5): "))
    
    return {
        'genre_weight': genre_weight,
        'director_weight': director_weight,
        'actor_weight': actor_weight,
        'keyword_weight': keyword_weight,
        'rating_weight': rating_weight,
        'overview_weight': overview_weight,
        'release_date_weight': release_date_weight,
        'votes_weight': votes_weight,
        'runtime_weight': runtime_weight,
        'language_weight': language_weight,
        'country_weight': country_weight,
        'certification_weight': certification_weight,
        'soundtrack_weight': soundtrack_weight,
        'production_companies_weight': production_companies_weight,
        'alternate_titles_weight': alternate_titles_weight,
        'tagline_weight': tagline_weight,
        'box_office_weight': box_office_weight
    }

def get_movie_by_name(movie_name):
    search_results = ia.search_movie(movie_name)
    if search_results:
        return ia.get_movie(search_results[0].movieID)
    return None

def recommend_similar_movies(movie, user_weights, num_recommendations=3):
    recommendations = []
    movie_list = ia.get_top250_movies()  # Example: getting a list of top movies. Adjust as needed.
    for other_movie in movie_list:
        if other_movie.movieID == movie.movieID:
            continue
        
        score = calculate_similarity(movie, other_movie, user_weights)
        recommendations.append((other_movie, score))
    
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:num_recommendations]

def recommend_movies(movie_name):
    user_weights = get_user_preferences()
    movie = get_movie_by_name(movie_name)
    if not movie:
        return "Movie not found. Please try a different name."
    
    print(f"Movie found: {movie['title']} ({movie['year']})")

    recommendations = recommend_similar_movies(movie, user_weights)
    if recommendations:
        print("\nRecommended movies:")
        for rec_movie, score in recommendations:
            print(f"- {rec_movie['title']} ({rec_movie['year']}) | Rating: {rec_movie.get('rating', 'N/A')} | Score: {score:.2f}")
    else:
        print("No similar movies found.")

# Example usage
movie_name = input("Enter a movie name: ")
recommend_movies(movie_name)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter a movie name:  interstellar


Assign weights to the following features (0-5):


Genres (0-5):  3
Directors (0-5):  5
Actors (0-5):  1
Keywords (0-5):  3
Ratings (0-5):  5
Overview (0-5):  5
Release Date (0-5):  0
Votes (0-5):  2
Runtime (0-5):  0
Language (0-5):  0
Country (0-5):  0
Certification (0-5):  3
Soundtrack (0-5):  3
Production Companies (0-5):  0
Alternate Titles (0-5):  0
Tagline (0-5):  0
Box Office (0-5):  3


Movie found: Interstellar (2014)
No similar movies found.


In [6]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Load the dataset
file_path = '/kaggle/input/movies-updated/movies_with_tags_rating.csv'
df = pd.read_csv(file_path)


# Fill missing values
df['title'].fillna('No title')
df['film_rate'].fillna('Not Mentioned')  # Adjust as needed
df['genre'].fillna('Not Mentioned')
df['cast'].fillna('Not Mentioned')
df['reviews'].fillna('Not Mentioned')  # Adjust as needed
df['imdbId'].fillna('Not Mentioned')
df['director'].fillna('Not Mentioned')
df['rating'].fillna(0)  # Assuming rating is numerical
df['tag'].fillna('Not Mentioned')

"""
# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
"""
# Initialize NLTK components
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def get_movie_features(movie_title, df):
    movie = df[df['title'].str.lower() == movie_title.lower()]
    if movie.empty:
        return None
    movie = movie.iloc[0]  # Get the first row
    
    # Use .get() method or default to empty strings if NaN or missing
    return {
        'title': movie.get('title', ''),
        'year': movie.get('film_rate', ''),  # Adjust as needed
        'genres': str(movie.get('genre', '')).split(', '),  # Convert to string and split
        'directors': [str(movie.get('director', ''))],
        'actors': str(movie.get('cast', '')).split(', '),  # Convert to string and split
        'keywords': str(movie.get('tag', '')).split(', '),  # Convert to string and split
        'rating': movie.get('rating', 0),
        'overview': str(movie.get('overview', '')),
        'release_date': '',  # Add or adjust as needed
        'votes': 0,  # Add or adjust as needed
        'runtime': 0,  # Add or adjust as needed
        'language': '',  # Add or adjust as needed
        'country': '',  # Add or adjust as needed
        'certification': '',  # Add or adjust as needed
        'soundtrack': '',  # Add or adjust as needed
        'production_companies': '',  # Add or adjust as needed
        'alternate_titles': '',  # Add or adjust as needed
        'tagline': '',  # Add or adjust as needed
        'box_office': {}  # Add or adjust as needed
    }

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

def calculate_text_similarity(text1, text2):
    # Placeholder for text similarity implementation
    return 1 if text1.lower() == text2.lower() else 0

def calculate_date_similarity(date1, date2):
    return 1 if date1[:4] == date2[:4] else 0

def calculate_box_office_similarity(box_office1, box_office2):
    return 1 - abs(box_office1.get('gross', 0) - box_office2.get('gross', 0)) / max(box_office1.get('gross', 1), box_office2.get('gross', 1))

def calculate_similarity(movie, other_movie, user_weights):
    movie_features = get_movie_features(movie, df)
    other_features = get_movie_features(other_movie, df)
    
    if not movie_features or not other_features:
        return 0  # Return 0 if either movie is not found

    genre_similarity = len(set(movie_features['genres']) & set(other_features['genres'])) * user_weights['genre_weight']
    director_similarity = len(set(movie_features['directors']) & set(other_features['directors'])) * user_weights['director_weight']
    actor_similarity = len(set(movie_features['actors']) & set(other_features['actors'])) * user_weights['actor_weight']
    keyword_similarity = len(set(movie_features['keywords']) & set(other_features['keywords'])) * user_weights['keyword_weight']
    rating_similarity = (1 - abs(movie_features['rating'] - other_features['rating']) / 10) * user_weights['rating_weight']
    overview_similarity = calculate_text_similarity(preprocess_text(movie_features['overview']), preprocess_text(other_features['overview'])) * user_weights['overview_weight']
    release_date_similarity = calculate_date_similarity(movie_features['release_date'], other_features['release_date']) * user_weights['release_date_weight']
    votes_similarity = (1 - abs(movie_features['votes'] - other_features['votes']) / max(movie_features['votes'], 1)) * user_weights['votes_weight']
    runtime_similarity = (1 - abs(movie_features['runtime'] - other_features['runtime']) / max(movie_features['runtime'], 1)) * user_weights['runtime_weight']
    language_similarity = len(set(movie_features['language']) & set(other_features['language'])) * user_weights['language_weight']
    country_similarity = len(set(movie_features['country']) & set(other_features['country'])) * user_weights['country_weight']
    certification_similarity = len(set(movie_features['certification']) & set(other_features['certification'])) * user_weights['certification_weight']
    soundtrack_similarity = len(set(movie_features['soundtrack']) & set(other_features['soundtrack'])) * user_weights['soundtrack_weight']
    production_companies_similarity = len(set(movie_features['production_companies']) & set(other_features['production_companies'])) * user_weights['production_companies_weight']
    alternate_titles_similarity = len(set(movie_features['alternate_titles']) & set(other_features['alternate_titles'])) * user_weights['alternate_titles_weight']
    tagline_similarity = calculate_text_similarity(movie_features['tagline'], other_features['tagline']) * user_weights['tagline_weight']
    box_office_similarity = calculate_box_office_similarity(movie_features['box_office'], other_features['box_office']) * user_weights['box_office_weight']
    
    return (genre_similarity + director_similarity + actor_similarity + keyword_similarity + rating_similarity +
            overview_similarity + release_date_similarity + votes_similarity + runtime_similarity +
            language_similarity + country_similarity + certification_similarity + soundtrack_similarity +
            production_companies_similarity + alternate_titles_similarity + tagline_similarity +
            box_office_similarity)

def get_user_preferences():
    print("Assign weights to the following features (0-5):")
    genre_weight = int(input("Genres (0-5): "))
    director_weight = int(input("Directors (0-5): "))
    actor_weight = int(input("Actors (0-5): "))
    keyword_weight = int(input("Keywords (0-5): "))
    rating_weight = int(input("Ratings (0-5): "))
    overview_weight = int(input("Overview (0-5): "))
    release_date_weight = int(input("Release Date (0-5): "))
    votes_weight = int(input("Votes (0-5): "))
    runtime_weight = int(input("Runtime (0-5): "))
    language_weight = int(input("Language (0-5): "))
    country_weight = int(input("Country (0-5): "))
    certification_weight = int(input("Certification (0-5): "))
    soundtrack_weight = int(input("Soundtrack (0-5): "))
    production_companies_weight = int(input("Production Companies (0-5): "))
    alternate_titles_weight = int(input("Alternate Titles (0-5): "))
    tagline_weight = int(input("Tagline (0-5): "))
    box_office_weight = int(input("Box Office (0-5): "))
    
    return {
        'genre_weight': genre_weight,
        'director_weight': director_weight,
        'actor_weight': actor_weight,
        'keyword_weight': keyword_weight,
        'rating_weight': rating_weight,
        'overview_weight': overview_weight,
        'release_date_weight': release_date_weight,
        'votes_weight': votes_weight,
        'runtime_weight': runtime_weight,
        'language_weight': language_weight,
        'country_weight': country_weight,
        'certification_weight': certification_weight,
        'soundtrack_weight': soundtrack_weight,
        'production_companies_weight': production_companies_weight,
        'alternate_titles_weight': alternate_titles_weight,
        'tagline_weight': tagline_weight,
        'box_office_weight': box_office_weight
    }

def recommend_similar_movies(movie, user_weights, df):
    # Get all movies
    all_movies = df['title'].tolist()
    
    # Calculate similarity for all movies
    similarities = []
    for other_movie in all_movies:
        if other_movie == movie:
            continue
        other_movie_features = get_movie_features(other_movie, df)
        if other_movie_features:
            score = calculate_similarity(movie, other_movie_features, user_weights)
            similarities.append((other_movie_features, score))
    
    # Sort movies by similarity score
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return similarities[:10]  # Return top 10 recommendations

def recommend_movies(movie_name):
    user_weights = get_user_preferences()
    movie = get_movie_features(movie_name, df)
    if not movie:
        print("Movie not found. Please try a different name.")
        return

    print(f"Movie found: {movie['title']} ({movie['year']})")

    recommendations = recommend_similar_movies(movie, user_weights, df)
    if recommendations:
        print("\nRecommended movies:")
        for rec_movie, score in recommendations:
            print(f"- {rec_movie['title']} ({rec_movie['year']}) | Rating: {rec_movie.get('rating', 'N/A')} | Score: {score:.2f}")
    else:
        print("No similar movies found.")

# Example usage
movie_name = input("Enter a movie name: ")
recommend_movies(movie_name)


Enter a movie name:  heat


Assign weights to the following features (0-5):


Genres (0-5):  3
Directors (0-5):  3
Actors (0-5):  3
Keywords (0-5):  3
Ratings (0-5):  5
Overview (0-5):  5
Release Date (0-5):  0
Votes (0-5):  3
Runtime (0-5):  0
Language (0-5):  0
Country (0-5):  0
Certification (0-5):  0
Soundtrack (0-5):  0
Production Companies (0-5):  0
Alternate Titles (0-5):  0
Tagline (0-5):  0
Box Office (0-5):  0


Movie found: Heat (8.3)


AttributeError: 'dict' object has no attribute 'lower'

In [7]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Load the dataset
file_path = '/kaggle/input/movies-updated/movies_with_tags_rating.csv'
df = pd.read_csv(file_path)

# Fill missing values
df['title'].fillna('No title', inplace=True)
df['film_rate'].fillna('Not Mentioned', inplace=True)  # Adjust as needed
df['genre'].fillna('Not Mentioned', inplace=True)
df['cast'].fillna('Not Mentioned', inplace=True)
df['reviews'].fillna('Not Mentioned', inplace=True)  # Adjust as needed
df['imdbId'].fillna('Not Mentioned', inplace=True)
df['director'].fillna('Not Mentioned', inplace=True)
df['rating'].fillna(0, inplace=True)  # Assuming rating is numerical
df['tag'].fillna('Not Mentioned', inplace=True)

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize NLTK components
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def get_movie_features(movie_title, df):
    movie = df[df['title'].str.lower() == movie_title.lower()]
    if movie.empty:
        return None
    movie = movie.iloc[0]  # Get the first row
    
    return {
        'title': movie.get('title', 'Not Mentioned'),
        'year': movie.get('film_rate', 'Not Mentioned'),  # Adjust as needed
        'genres': str(movie.get('genre', 'Not Mentioned')).split(', '),
        'directors': [str(movie.get('director', 'Not Mentioned'))],
        'actors': str(movie.get('cast', 'Not Mentioned')).split(', '),
        'keywords': str(movie.get('tag', 'Not Mentioned')).split(', '),
        'rating': movie.get('rating', 0),
        'overview': str(movie.get('overview', 'Not Mentioned')),
        'release_date': '',  # Add or adjust as needed
        'votes': 0,  # Add or adjust as needed
        'runtime': 0,  # Add or adjust as needed
        'language': '',  # Add or adjust as needed
        'country': '',  # Add or adjust as needed
        'certification': '',  # Add or adjust as needed
        'soundtrack': '',  # Add or adjust as needed
        'production_companies': '',  # Add or adjust as needed
        'alternate_titles': '',  # Add or adjust as needed
        'tagline': '',  # Add or adjust as needed
        'box_office': {}  # Add or adjust as needed
    }

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

def calculate_text_similarity(text1, text2):
    # Placeholder for text similarity implementation
    return 1 if text1.lower() == text2.lower() else 0

def calculate_date_similarity(date1, date2):
    return 1 if date1[:4] == date2[:4] else 0

def calculate_box_office_similarity(box_office1, box_office2):
    return 1 - abs(box_office1.get('gross', 0) - box_office2.get('gross', 0)) / max(box_office1.get('gross', 1), box_office2.get('gross', 1))

def calculate_similarity(movie, other_movie, user_weights):
    movie_features = get_movie_features(movie, df)
    other_features = get_movie_features(other_movie, df)
    
    if not movie_features or not other_features:
        return 0  # Return 0 if either movie is not found

    genre_similarity = len(set(movie_features['genres']) & set(other_features['genres'])) * user_weights['genre_weight']
    director_similarity = len(set(movie_features['directors']) & set(other_features['directors'])) * user_weights['director_weight']
    actor_similarity = len(set(movie_features['actors']) & set(other_features['actors'])) * user_weights['actor_weight']
    keyword_similarity = len(set(movie_features['keywords']) & set(other_features['keywords'])) * user_weights['keyword_weight']
    rating_similarity = (1 - abs(movie_features['rating'] - other_features['rating']) / 10) * user_weights['rating_weight']
    overview_similarity = calculate_text_similarity(preprocess_text(movie_features['overview']), preprocess_text(other_features['overview'])) * user_weights['overview_weight']
    release_date_similarity = calculate_date_similarity(movie_features['release_date'], other_features['release_date']) * user_weights['release_date_weight']
    votes_similarity = (1 - abs(movie_features['votes'] - other_features['votes']) / max(movie_features['votes'], 1)) * user_weights['votes_weight']
    runtime_similarity = (1 - abs(movie_features['runtime'] - other_features['runtime']) / max(movie_features['runtime'], 1)) * user_weights['runtime_weight']
    language_similarity = len(set(movie_features['language']) & set(other_features['language'])) * user_weights['language_weight']
    country_similarity = len(set(movie_features['country']) & set(other_features['country'])) * user_weights['country_weight']
    certification_similarity = len(set(movie_features['certification']) & set(other_features['certification'])) * user_weights['certification_weight']
    soundtrack_similarity = len(set(movie_features['soundtrack']) & set(other_features['soundtrack'])) * user_weights['soundtrack_weight']
    production_companies_similarity = len(set(movie_features['production_companies']) & set(other_features['production_companies'])) * user_weights['production_companies_weight']
    alternate_titles_similarity = len(set(movie_features['alternate_titles']) & set(other_features['alternate_titles'])) * user_weights['alternate_titles_weight']
    tagline_similarity = calculate_text_similarity(movie_features['tagline'], other_features['tagline']) * user_weights['tagline_weight']
    box_office_similarity = calculate_box_office_similarity(movie_features['box_office'], other_features['box_office']) * user_weights['box_office_weight']
    
    return (genre_similarity + director_similarity + actor_similarity + keyword_similarity + rating_similarity +
            overview_similarity + release_date_similarity + votes_similarity + runtime_similarity +
            language_similarity + country_similarity + certification_similarity + soundtrack_similarity +
            production_companies_similarity + alternate_titles_similarity + tagline_similarity +
            box_office_similarity)

def get_user_preferences():
    print("Assign weights to the following features (0-5):")
    genre_weight = int(input("Genres (0-5): "))
    director_weight = int(input("Directors (0-5): "))
    actor_weight = int(input("Actors (0-5): "))
    keyword_weight = int(input("Keywords (0-5): "))
    rating_weight = int(input("Ratings (0-5): "))
    overview_weight = int(input("Overview (0-5): "))
    release_date_weight = int(input("Release Date (0-5): "))
    votes_weight = int(input("Votes (0-5): "))
    runtime_weight = int(input("Runtime (0-5): "))
    language_weight = int(input("Language (0-5): "))
    country_weight = int(input("Country (0-5): "))
    certification_weight = int(input("Certification (0-5): "))
    soundtrack_weight = int(input("Soundtrack (0-5): "))
    production_companies_weight = int(input("Production Companies (0-5): "))
    alternate_titles_weight = int(input("Alternate Titles (0-5): "))
    tagline_weight = int(input("Tagline (0-5): "))
    box_office_weight = int(input("Box Office (0-5): "))
    
    return {
        'genre_weight': genre_weight,
        'director_weight': director_weight,
        'actor_weight': actor_weight,
        'keyword_weight': keyword_weight,
        'rating_weight': rating_weight,
        'overview_weight': overview_weight,
        'release_date_weight': release_date_weight,
        'votes_weight': votes_weight,
        'runtime_weight': runtime_weight,
        'language_weight': language_weight,
        'country_weight': country_weight,
        'certification_weight': certification_weight,
        'soundtrack_weight': soundtrack_weight,
        'production_companies_weight': production_companies_weight,
        'alternate_titles_weight': alternate_titles_weight,
        'tagline_weight': tagline_weight,
        'box_office_weight': box_office_weight
    }

def recommend_similar_movies(movie, user_weights, df):
    # Get all movies
    all_movies = df['title'].tolist()
    
    # Calculate similarity for all movies
    similarities = []
    for other_movie in all_movies:
        if other_movie == movie:
            continue
        other_movie_features = get_movie_features(other_movie, df)
        if other_movie_features:
            score = calculate_similarity(movie, other_movie, user_weights)
            similarities.append((other_movie_features, score))
    
    # Sort movies by similarity score
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return similarities[:10]  # Return top 10 recommendations

def recommend_movies(movie_name):
    user_weights = get_user_preferences()
    movie = get_movie_features(movie_name, df)
    if not movie:
        print("Movie not found. Please try a different name.")
        return

    print(f"Movie found: {movie['title']} ({movie['year']})")

    recommendations = recommend_similar_movies(movie['title'], user_weights, df)
    if recommendations:
        print("\nRecommended movies:")
        for rec_movie, score in recommendations:
            print(f"- {rec_movie['title']} ({rec_movie['year']}) | Rating: {rec_movie.get('rating', 'N/A')} | Score: {score:.2f}")
    else:
        print("No similar movies found.")

# Example usage
movie_name = input("Enter a movie name: ")
recommend_movies(movie_name)


/tmp/ipykernel_158/1561368314.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['title'].fillna('No title', inplace=True)
/tmp/ipykernel_158/1561368314.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter a movie name:  heat


Assign weights to the following features (0-5):


Genres (0-5):  3
Directors (0-5):  5
Actors (0-5):  1
Keywords (0-5):  5
Ratings (0-5):  5
Overview (0-5):  5
Release Date (0-5):  0
Votes (0-5):  0
Runtime (0-5):  0
Language (0-5):  0
Country (0-5):  0
Certification (0-5):  0
Soundtrack (0-5):  0
Production Companies (0-5):  0
Alternate Titles (0-5):  0
Tagline (0-5):  0
Box Office (0-5):  0


Movie found: Heat (8.3)


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [1]:
pip install tmdbv3api


Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd

user_movie_ratings = pd.read_csv('/kaggle/input/rating-imdbid/ratings.csv')
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Check the first few rows of each DataFrame to ensure proper loading
print(user_movie_ratings.head())
print(movies_with_tags.head())

# Merge datasets on 'imdbId'
user_movie_ratings = user_movie_ratings.merge(movies_with_tags[['imdbId', 'title', 'director', 'genre', 'rating']], on='imdbId', how='left')

# Now user_movie_ratings contains additional movie details from movies_with_tags
print(user_movie_ratings.head())


   userId  imdbId  rating   timestamp
0       1       1     4.0  1225734739
1       1     110     4.0  1225865086
2       1     158     4.0  1225733503
3       1     260     4.5  1225735204
4       1     356     5.0  1225735119
   index                          title  film_rate  \
0      0               Before and After        6.1   
1      1          Anne Frank Remembered        8.1   
2      2  The Young Poisoner's Handbook        7.0   
3      3                   If Lucy Fell        4.9   
4      4         Steal Big Steal Little        4.7   

                                            overview  \
0  ['Two parents deal with the effects when their...   
1  ["Tells the story of the Frank family and pain...   
2  ["This film is based on a true story about a B...   
3  ["Two NYC roommates have a pact to jump off Br...   
4  ["Ruben and Robby are twin brothers, adopted b...   

                                           cover_url  \
0  https://m.media-amazon.com/images/M/MV5BOWJmOD...  

In [15]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the ratings dataset with IMDb IDs
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)

# Load the movie titles dataset (with IMDb IDs)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Merge ratings with movie details
user_movie_ratings = user_movie_ratings.merge(movies_with_tags[['imdbId', 'title', 'director', 'genre']], on='imdbId', how='left')

# Function to fetch genres and director for a movie using TMDb (searching by TMDb ID)
def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details including credits
    movie_details = movie.details(tmdb_id, append_to_response='credits')
    
    # Extract genres
    genres = [g['name'] for g in movie_details.genres]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.credits['crew']:
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
            
    return genres, director

# Collaborative Filtering (using matrix factorization)
def collaborative_filtering(user_movie_ratings):
    # User-Item Matrix (using movie titles instead of movie IDs)
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Matrix Factorization using Truncated SVD
    svd = TruncatedSVD(n_components=300)  # Increase latent factors to 300
    matrix_factorized = svd.fit_transform(user_movie_matrix)

    # Cosine Similarity between movies
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    return cosine_sim_matrix, user_movie_matrix.columns

def recommend_movies(movie_name, user_movie_ratings):
    # Search for the movie by name
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    # Get the first search result's TMDb ID
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    # Fetch movie details (genres, director) using TMDb ID
    genres, director = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}")
    
    # Check if IMDb ID exists in the ratings dataset
    imdb_id = search_results[0].id
    if imdb_id not in user_movie_ratings['imdbId'].unique():
        return "Movie not found in the ratings dataset."

    # Get the user-item matrix and perform collaborative filtering
    cosine_sim_matrix, movie_titles = collaborative_filtering(user_movie_ratings)
    
    # Find the movie index in the cosine similarity matrix (title to index mapping)
    try:
        movie_idx = list(movie_titles).index(search_results[0].title)
    except ValueError:
        return "Movie not found in the ratings dataset."

    # Find similar movies using collaborative filtering
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))

    # Sort by similarity score
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    # Fetch details for top similar movies
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = movie_titles[movie_idx]
        
        # Filter by content (same genres, same director)
        similar_movie_id = movies_with_tags[movies_with_tags['title'] == similar_movie_title]['imdbId'].values[0]
        rec_genres, rec_director = get_movie_details_by_tmdb(similar_movie_id)
        
        if any(g in genres for g in rec_genres) or rec_director == director:
            recommended_movies.append(f"{i+1}. {similar_movie_title} (Genres: {rec_genres})")
            if len(recommended_movies) == 3:
                break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name, user_movie_ratings))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: Michael Mann


ValueError: Index contains duplicate entries, cannot reshape

In [16]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the ratings dataset with IMDb IDs
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)

# Load the movie titles dataset (with IMDb IDs)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Merge ratings with movie details
user_movie_ratings = user_movie_ratings.merge(movies_with_tags[['imdbId', 'title', 'director', 'genre']], on='imdbId', how='left')

# Function to fetch genres and director for a movie using TMDb (searching by TMDb ID)
def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details including credits
    movie_details = movie.details(tmdb_id, append_to_response='credits')
    
    # Extract genres
    genres = [g['name'] for g in movie_details.genres]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.credits['crew']:
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
            
    return genres, director

# Collaborative Filtering (using matrix factorization)
def collaborative_filtering(user_movie_ratings):
    # Remove duplicates by averaging ratings for user-movie pairs
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title']).rating.mean().reset_index()

    # User-Item Matrix (using movie titles instead of movie IDs)
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Matrix Factorization using Truncated SVD
    svd = TruncatedSVD(n_components=300)  # Increase latent factors to 300
    matrix_factorized = svd.fit_transform(user_movie_matrix)

    # Cosine Similarity between movies
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    return cosine_sim_matrix, user_movie_matrix.columns

def recommend_movies(movie_name, user_movie_ratings):
    # Search for the movie by name
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    # Get the first search result's TMDb ID
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    # Fetch movie details (genres, director) using TMDb ID
    genres, director = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}")
    
    # Check if IMDb ID exists in the ratings dataset
    imdb_id = search_results[0].id
    if imdb_id not in user_movie_ratings['imdbId'].unique():
        return "Movie not found in the ratings dataset."

    # Get the user-item matrix and perform collaborative filtering
    cosine_sim_matrix, movie_titles = collaborative_filtering(user_movie_ratings)
    
    # Find the movie index in the cosine similarity matrix (title to index mapping)
    try:
        movie_idx = list(movie_titles).index(search_results[0].title)
    except ValueError:
        return "Movie not found in the ratings dataset."

    # Find similar movies using collaborative filtering
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))

    # Sort by similarity score
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    # Fetch details for top similar movies
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = movie_titles[movie_idx]
        
        # Filter by content (same genres, same director)
        similar_movie_id = movies_with_tags[movies_with_tags['title'] == similar_movie_title]['imdbId'].values[0]
        rec_genres, rec_director = get_movie_details_by_tmdb(similar_movie_id)
        
        if any(g in genres for g in rec_genres) or rec_director == director:
            recommended_movies.append(f"{i+1}. {similar_movie_title} (Genres: {rec_genres})")
            if len(recommended_movies) == 3:
                break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name, user_movie_ratings))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: Michael Mann
Movie not found in the ratings dataset.


In [3]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the ratings dataset with IMDb IDs
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)

# Load the movie titles dataset (with IMDb IDs)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Merge ratings with movie details
user_movie_ratings = user_movie_ratings.merge(movies_with_tags[['imdbId', 'title', 'director', 'genre', 'overview']], on='imdbId', how='left')

# Function to fetch genres and director for a movie using TMDb (searching by TMDb ID)
def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details including credits
    movie_details = movie.details(tmdb_id, append_to_response='credits')
    
    # Extract genres
    genres = [g['name'] for g in movie_details.genres]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.credits['crew']:
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
            
    return genres, director

# Collaborative Filtering (using matrix factorization)
def collaborative_filtering(user_movie_ratings):
    # Aggregate duplicate entries by taking the mean rating
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    # User-Item Matrix (using movie titles instead of movie IDs)
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Matrix Factorization using Truncated SVD
    svd = TruncatedSVD(n_components=300)  # Increase latent factors to 300
    matrix_factorized = svd.fit_transform(user_movie_matrix)

    # Cosine Similarity between movies
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    return cosine_sim_matrix, user_movie_matrix.columns


def recommend_movies_by_overview(movie_name):
    # Get the movie details
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    # Get the first search result's TMDb ID and details
    tmdb_id = search_results[0].id
    movie_details = movie.details(tmdb_id)
    movie_overview = movie_details.overview
    
    # Use TF-IDF to vectorize overviews and compute similarity
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_with_tags['overview'].fillna(''))
    movie_tfidf = tfidf_vectorizer.transform([movie_overview])
    
    # Compute cosine similarity between the input movie and all movies in the dataset
    cosine_similarities = cosine_similarity(movie_tfidf, tfidf_matrix).flatten()
    
    # Get the indices of the top similar movies
    similar_indices = cosine_similarities.argsort()[-10:][::-1]
    similar_movies = movies_with_tags.iloc[similar_indices]
    
    recommended_movies = []
    for idx in similar_movies.index:
        recommended_movies.append(f"{idx+1}. {similar_movies.iloc[idx]['title']} (Overview: {similar_movies.iloc[idx]['overview'][:100]}...)")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies(movie_name, user_movie_ratings):
    # Search for the movie by name
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    # Get the first search result's TMDb ID
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    # Fetch movie details (genres, director) using TMDb ID
    genres, director = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}")
    
    # Check if IMDb ID exists in the ratings dataset
    imdb_id = search_results[0].id
    if imdb_id not in user_movie_ratings['imdbId'].unique():
        # Recommend based on overview if the movie is not found in the dataset
        return recommend_movies_by_overview(movie_name)

    # Get the user-item matrix and perform collaborative filtering
    cosine_sim_matrix, movie_titles = collaborative_filtering(user_movie_ratings)
    
    # Find the movie index in the cosine similarity matrix (title to index mapping)
    try:
        movie_idx = list(movie_titles).index(search_results[0].title)
    except ValueError:
        return "Movie not found in the ratings dataset."

    # Find similar movies using collaborative filtering
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))

    # Sort by similarity score
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    # Fetch details for top similar movies
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = movie_titles[movie_idx]
        
        # Filter by content (same genres, same director)
        similar_movie_id = movies_with_tags[movies_with_tags['title'] == similar_movie_title]['imdbId'].values[0]
        rec_genres, rec_director = get_movie_details_by_tmdb(similar_movie_id)
        
        if any(g in genres for g in rec_genres) or rec_director == director:
            recommended_movies.append(f"{i+1}. {similar_movie_title} (Genres: {rec_genres})")
            if len(recommended_movies) == 3:
                break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name, user_movie_ratings))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: Michael Mann
Movie not found in the ratings dataset.


In [2]:
pip install tmdbv3api

Note: you may need to restart the kernel to use updated packages.


In [7]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the ratings dataset with IMDb IDs
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)

# Load the movie titles dataset (with IMDb IDs)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Merge ratings with movie details
user_movie_ratings = user_movie_ratings.merge(movies_with_tags[['imdbId', 'title', 'director', 'genre', 'overview']], on='imdbId', how='left')

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
            
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def collaborative_filtering(user_movie_ratings):
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    svd = TruncatedSVD(n_components=300)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    return cosine_sim_matrix, user_movie_matrix.columns

def recommend_movies_by_overview(movie_name):
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    movie_details = movie.details(tmdb_id)
    movie_overview = movie_details.get('overview', '')
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_with_tags['overview'].fillna(''))
    movie_tfidf = tfidf_vectorizer.transform([movie_overview])
    
    cosine_similarities = cosine_similarity(movie_tfidf, tfidf_matrix).flatten()
    
    similar_indices = cosine_similarities.argsort()[-10:][::-1]
    similar_movies = movies_with_tags.iloc[similar_indices]
    
    recommended_movies = []
    for idx in similar_movies.index:
        recommended_movies.append(f"{idx+1}. {similar_movies.iloc[idx]['title']} (Overview: {similar_movies.iloc[idx]['overview'][:100]}...)")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies(movie_name, user_movie_ratings):
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")
    
    if imdb_id not in user_movie_ratings['imdbId'].astype(str).unique():
        return recommend_movies_by_overview(movie_name)

    cosine_sim_matrix, movie_titles = collaborative_filtering(user_movie_ratings)
    
    try:
        movie_idx = list(movie_titles).index(search_results[0].title)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = movie_titles[movie_idx]
        
        similar_movie_id = movies_with_tags[movies_with_tags['title'] == similar_movie_title]['imdbId'].values[0]
        rec_genres, rec_director, _ = get_movie_details_by_tmdb(similar_movie_id)
        
        if any(g in genres for g in rec_genres) or rec_director == director:
            recommended_movies.append(f"{i+1}. {similar_movie_title} (Genres: {rec_genres})")
            if len(recommended_movies) == 3:
                break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name, user_movie_ratings))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: None, IMDb ID: tt0113277


IndexError: single positional indexer is out-of-bounds

In [11]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the ratings dataset with IMDb IDs
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)

# Load the movie titles dataset (with IMDb IDs)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Merge ratings with movie details
user_movie_ratings = user_movie_ratings.merge(movies_with_tags[['imdbId', 'title', 'director', 'genre', 'overview']], on='imdbId', how='left')

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def collaborative_filtering(user_movie_ratings):
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    svd = TruncatedSVD(n_components=300)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    return cosine_sim_matrix, user_movie_matrix.columns

def recommend_movies_by_overview(movie_name):
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    movie_details = movie.details(tmdb_id)
    movie_overview = movie_details.get('overview', '')
    
    # Check if overview is empty
    if not movie_overview:
        return "No overview found for the movie."

    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_with_tags['overview'].fillna(''))
    movie_tfidf = tfidf_vectorizer.transform([movie_overview])
    
    cosine_similarities = cosine_similarity(movie_tfidf, tfidf_matrix).flatten()
    
    # Ensure we have similarities calculated
    if not cosine_similarities.any():
        return "Similarity calculation failed."

    # Get indices of the most similar movies
    similar_indices = cosine_similarities.argsort()[-10:][::-1]
    
    # Handle case where similar_indices may exceed DataFrame length
    similar_indices = [idx for idx in similar_indices if idx < len(movies_with_tags)]
    
    if not similar_indices:
        return "No similar movies found."

    similar_movies = movies_with_tags.iloc[similar_indices]
    
    # Debug: print similar_movies to inspect
    print("Similar Movies DataFrame:")
    print(similar_movies)
    
    recommended_movies = []
    for idx in similar_indices:
        if idx >= len(similar_movies) or idx < 0:
            continue  # Skip indices that are out-of-bounds
        movie_title = similar_movies.iloc[idx]['title']
        recommended_movies.append(f"{movie_title}")
        if len(recommended_movies) == 3:
            break
    
    if not recommended_movies:
        return "No recommendations found."
    
    return "\n".join(recommended_movies)



def recommend_movies(movie_name, user_movie_ratings):
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")
    
    if imdb_id not in user_movie_ratings['imdbId'].astype(str).unique():
        return recommend_movies_by_overview(movie_name)

    cosine_sim_matrix, movie_titles = collaborative_filtering(user_movie_ratings)
    
    try:
        movie_idx = list(movie_titles).index(search_results[0].title)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = movie_titles[movie_idx]
        
        similar_movie_id = movies_with_tags[movies_with_tags['title'] == similar_movie_title]['imdbId'].values[0]
        rec_genres, rec_director, _ = get_movie_details_by_tmdb(similar_movie_id)
        
        if any(g in genres for g in rec_genres) or rec_director == director:
            recommended_movies.append(f"{i+1}. {similar_movie_title} (Genres: {rec_genres})")
            if len(recommended_movies) == 3:
                break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."


# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name, user_movie_ratings))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: None, IMDb ID: tt0113277
Similar Movies DataFrame:
      index            title  film_rate  \
114     114             Heat        8.3   
7834   7834       The Reader        7.6   
8468   8468            Hanna        6.7   
6175   6175       Collateral        7.5   
4724   4724  City by the Sea        6.1   
9395   9395      St. Vincent        7.2   
1531   1531        Mousehunt        6.5   
4538   4538         Time Out        7.3   
6080   6080      The Hitcher        5.6   
6081   6081      The Hitcher        5.6   

                                               overview  \
114   ['A group of high-end professional thieves sta...   
7834  ['Post-WWII Germany: Nearly a decade after his...   
8468  ['A sixteen-year-old girl who was raised by he...   
6175  ['A cab driver finds himself the hostage of an...   
4724  ['Vincent Lamarca, whose father was executed f...   
9395  ['A young boy whose parents have j

In [15]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the ratings dataset with IMDb IDs
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)

# Load the movie titles dataset (with IMDb IDs)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Normalize the movie titles by removing year and converting to lowercase
def normalize_title(title):
    if isinstance(title, str):
        title = title.lower().strip()
        title = re.sub(r'\(\d{4}\)', '', title).strip()  # Remove year in parentheses
        return title
    return ''

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def collaborative_filtering(user_movie_ratings):
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    svd = TruncatedSVD(n_components=300)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    return cosine_sim_matrix, user_movie_matrix.columns

def recommend_movies_by_overview(movie_name):
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    movie_details = movie.details(tmdb_id)
    movie_overview = movie_details.get('overview', '')
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_with_tags['overview'].fillna(''))
    movie_tfidf = tfidf_vectorizer.transform([movie_overview])
    
    cosine_similarities = cosine_similarity(movie_tfidf, tfidf_matrix).flatten()
    
    # Get indices of the most similar movies
    similar_indices = cosine_similarities.argsort()[-10:][::-1]
    similar_movies = movies_with_tags.iloc[similar_indices]
    
    recommended_movies = []
    for idx in similar_indices:
        if idx >= len(similar_movies) or idx < 0:
            continue  # Skip indices that are out-of-bounds
        movie_title = similar_movies.iloc[idx]['title']
        recommended_movies.append(f"{movie_title}")
        if len(recommended_movies) == 3:
            break
    
    if not recommended_movies:
        return "No recommendations found."
    
    return "\n".join(recommended_movies)

def recommend_movies(movie_name, user_movie_ratings):
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    movie_title = search_results[0].title
    normalized_title = normalize_title(movie_title)
    print(f"Found movie: {movie_title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")
    
    if imdb_id not in user_movie_ratings['imdbId'].astype(str).unique():
        return recommend_movies_by_overview(movie_name)

    cosine_sim_matrix, movie_titles = collaborative_filtering(user_movie_ratings)
    
    normalized_movie_titles = [normalize_title(title) for title in movie_titles]
    
    try:
        movie_idx = normalized_movie_titles.index(normalized_title)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = movie_titles[movie_idx]
        
        similar_movie_id = movies_with_tags[movies_with_tags['title'] == similar_movie_title]['imdbId'].values[0]
        rec_genres, rec_director, _ = get_movie_details_by_tmdb(similar_movie_id)
        
        if any(g in genres for g in rec_genres) or rec_director == director:
            recommended_movies.append(f"{i+1}. {similar_movie_title}")
            if len(recommended_movies) == 3:
                break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name, user_movie_ratings))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: None, IMDb ID: tt0113277
No recommendations found.


In [17]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)

movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Normalize movie titles by removing years and converting to lowercase
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def recommend_movies(movie_name):
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        # Collaborative filtering
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            # Content-based filtering
            return recommend_movies_by_content(genres, director, movie_name)
    else:
        # Fallback content-based filtering if IMDb ID is not available
        return recommend_movies_by_content(genres, director, movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, movie_name):
    # Vectorize the movie overviews
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_with_tags['overview'].fillna(''))
    
    # Create a query vector for the input movie
    movie_overview = movie.search(movie_name)[0].overview if movie.search(movie_name) else ''
    query_vector = tfidf_vectorizer.transform([movie_overview])
    
    # Compute cosine similarities
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Get indices of the most similar movies
    similar_indices = cosine_similarities.argsort()[-10:][::-1]
    similar_movies = movies_with_tags.iloc[similar_indices]
    
    recommended_movies = []
    for idx in similar_indices:
        movie_title = similar_movies.iloc[idx]['title']
        movie_genres = similar_movies.iloc[idx]['genre'].split(',') if not pd.isna(similar_movies.iloc[idx]['genre']) else []
        movie_director = similar_movies.iloc[idx]['director'] if not pd.isna(similar_movies.iloc[idx]['director']) else None
        
        # Check if the movie has at least one matching genre or matching director
        if (genres and any(g in movie_genres for g in genres)) or director == movie_director:
            recommended_movies.append(movie_title)
            if len(recommended_movies) == 3:
                break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: None, IMDb ID: tt0113277


IndexError: single positional indexer is out-of-bounds

In [28]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)

movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Normalize movie titles by removing years and converting to lowercase
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

# Add normalized titles to the dataset
movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    # Here, we should map the normalized title back to the original titles.
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def recommend_movies(movie_name):
    # Normalize the input movie name
    normalized_movie_name = normalize_title(movie_name)
    
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        # Collaborative filtering
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            # Content-based filtering
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        # Fallback content-based filtering if IMDb ID is not available
        return recommend_movies_by_content(genres, director, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, movie_name):
    # Normalize the movie name for matching
    normalized_movie_name = normalize_title(movie_name)
    
    # Ensure director is a string
    if director is None:
        director = ''
    
    # Filter movies that match genres and director
    similar_movies = movies_with_tags[
        movies_with_tags['genre'].str.contains('|'.join(genres), na=False) &
        movies_with_tags['director'].str.contains(director, na=False)
    ]
    
    # Check if there are any movies to recommend
    if similar_movies.empty:
        return "No recommendations found."

    # Drop the movie already entered from the recommendations
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]

    # Check if there are any movies left to recommend
    if similar_movies.empty:
        return "No recommendations found."

    # Select top recommendations based on some criterion
    recommended_movies = similar_movies.head(3)['title'].tolist()

    if not recommended_movies:
        return "No recommendations found."
    
    return recommended_movies


# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


Enter a movie name:  interstellar


Found movie: Interstellar (TMDb ID: 157336)
Genres: ['Adventure', 'Drama', 'Science Fiction'], Director: None, IMDb ID: tt0816692
['Before and After', "The Young Poisoner's Handbook", 'The Boys of St. Vincent']


In [31]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Normalize movie titles by removing years and converting to lowercase
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

# Add normalized titles to the dataset
movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    # Map the normalized title back to the original titles.
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def recommend_movies(movie_name):
    # Normalize the input movie name
    normalized_movie_name = normalize_title(movie_name)
    
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        # Collaborative filtering
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            # Content-based filtering
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        # Fallback content-based filtering if IMDb ID is not available
        return recommend_movies_by_content(genres, director, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, movie_name):
    # Normalize the movie name for matching
    normalized_movie_name = normalize_title(movie_name)
    
    # Ensure director is a string
    if director is None:
        director = ''
    
    # Convert genres to a string for TF-IDF
    genres_str = ' '.join(genres)
    
    # Combine overview, genres, and director for content-based filtering
    movies_with_tags['combined_features'] = (
        movies_with_tags['overview'].fillna('') + ' ' +
        movies_with_tags['genre'].fillna('') + ' ' +
        movies_with_tags['director'].fillna('')
    )

    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(movies_with_tags['combined_features'])
    
    # TF-IDF Vectorization for the input movie
    input_movie_vector = vectorizer.transform([movie_name])
    
    # Compute cosine similarity between the input movie and dataset
    cosine_similarities = cosine_similarity(input_movie_vector, tfidf_matrix).flatten()
    
    # Get indices of the top 10 most similar movies
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]
    
    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]
    
    # Check if there are any movies left to recommend
    if similar_movies.empty:
        return "No recommendations found."

    recommended_movies = similar_movies['title'].tolist()
    
    if not recommended_movies:
        return "No recommendations found."
    
    return recommended_movies

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


Enter a movie name:  me before you


Found movie: Me Before You (TMDb ID: 296096)
Genres: ['Drama', 'Romance'], Director: None, IMDb ID: tt2674426
['Hardcore Henry', 'Austin Powers in Goldmember', 'Defending Your Life', 'Poker Night', "'night, Mother", "What's Up, Tiger Lily?", 'Dog Soldiers', "It's My Party", 'Nymphomaniac: Vol. II', 'The Dilemma']


In [32]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Normalize movie titles by removing years and converting to lowercase
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

# Add normalized titles to the dataset
movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    # Map the normalized title back to the original titles.
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def recommend_movies(movie_name):
    # Normalize the input movie name
    normalized_movie_name = normalize_title(movie_name)
    
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        # Collaborative filtering
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            # Content-based filtering
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        # Fallback content-based filtering if IMDb ID is not available
        return recommend_movies_by_content(genres, director, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."


def recommend_movies_by_content(genres, director, movie_name):
    # Normalize the movie name for matching
    normalized_movie_name = normalize_title(movie_name)
    
    # Ensure director is a string
    if director is None:
        director = ''
    
    # Combine overview, genres, and director for content-based filtering
    movies_with_tags['combined_features'] = (
        movies_with_tags['overview'].fillna('') + ' ' +
        movies_with_tags['genre'].fillna('') + ' ' +
        movies_with_tags['director'].fillna('')
    )
    
    # TF-IDF Vectorization for movie features
    vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    tfidf_matrix = vectorizer.fit_transform(movies_with_tags['combined_features'])
    
    # TF-IDF Vectorization for the input movie
    input_movie_features = ' '.join([f"genre: {', '.join(genres)}", f"director: {director}"])
    input_movie_vector = vectorizer.transform([input_movie_features])
    
    # Compute cosine similarity between the input movie and dataset
    cosine_similarities = cosine_similarity(input_movie_vector, tfidf_matrix).flatten()
    
    # Get indices of the top 10 most similar movies
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]
    
    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]
    
    # Check if there are any movies left to recommend
    if similar_movies.empty:
        return "No recommendations found."

    recommended_movies = similar_movies['title'].tolist()
    
    if not recommended_movies:
        return "No recommendations found."
    
    return recommended_movies


# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: None, IMDb ID: tt0113277
['Digging Up the Marrow', 'Pusher II', 'Batman: The Killing Joke', 'Swelter', 'A Chorus Line', 'Mojave', 'White Sands', 'Batman: The Dark Knight Returns, Part 1', 'One Tough Cop', 'Undisputed 2: Last Man Standing']


In [34]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Normalize movie titles by removing years and converting to lowercase
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

# Add normalized titles to the dataset
movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    # Map the normalized title back to the original titles.
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def recommend_movies(movie_name):
    # Normalize the input movie name
    normalized_movie_name = normalize_title(movie_name)
    
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        # Collaborative filtering
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            # Content-based filtering
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        # Fallback content-based filtering if IMDb ID is not available
        return recommend_movies_by_content(genres, director, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, movie_name):
    # Normalize the movie name for matching
    normalized_movie_name = normalize_title(movie_name)
    
    # Ensure director is a string
    if director is None:
        director = ''
    
    # Combine overview, genres, and director for content-based filtering
    movies_with_tags['combined_features'] = (
        movies_with_tags['overview'].fillna('') + ' ' +
        movies_with_tags['genre'].fillna('') + ' ' +
        movies_with_tags['director'].fillna('')
    )
    
    # TF-IDF Vectorization for movie features
    vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    tfidf_matrix = vectorizer.fit_transform(movies_with_tags['combined_features'])
    
    # TF-IDF Vectorization for the input movie
    input_movie_features = ' '.join([f"genre: {', '.join(genres)}", f"director: {director}"])
    input_movie_vector = vectorizer.transform([input_movie_features])
    
    # Compute cosine similarity between the input movie and dataset
    cosine_similarities = cosine_similarity(input_movie_vector, tfidf_matrix).flatten()
    
    # Get indices of the top 10 most similar movies
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]
    
    # Debugging output
    print("Cosine similarities:", cosine_similarities)
    print("Similar indices:", similar_indices)
    
    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]
    
    # Check if there are any movies left to recommend
    if similar_movies.empty:
        return "No recommendations found."

    recommended_movies = similar_movies['title'].tolist()
    
    if not recommended_movies:
        return "No recommendations found."
    
    return recommended_movies

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: None, IMDb ID: tt0113277
Cosine similarities: [0.01728618 0.         0.01614282 ... 0.00125946 0.00115076 0.00135964]
Similar indices: [ 9545  6874 10181 10020  5153 10002  3508  8905  2213  8146]
['Digging Up the Marrow', 'Pusher II', 'Batman: The Killing Joke', 'Swelter', 'A Chorus Line', 'Mojave', 'White Sands', 'Batman: The Dark Knight Returns, Part 1', 'One Tough Cop', 'Undisputed 2: Last Man Standing']


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel
import torch

# BERT tokenizer and model initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Extract features
movies_with_tags['overview_bert'] = movies_with_tags['overview'].apply(lambda x: bert_embedding(x))


In [ ]:
# Create TF-IDF for genres and director
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
genre_director_tfidf = tfidf_vectorizer.fit_transform(movies_with_tags['genre'] + ' ' + movies_with_tags['director'])

# Combine BERT embeddings with TF-IDF vectors
combined_features = torch.cat((torch.tensor(movies_with_tags['overview_bert'].tolist()), 
                                torch.tensor(genre_director_tfidf.toarray())), dim=1).numpy()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies_by_content(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    input_movie_features = movies_with_tags[movies_with_tags['normalized_title'] == normalized_movie_name]
    
    if input_movie_features.empty:
        return "No recommendations found."

    input_movie_vector = torch.tensor(bert_embedding(input_movie_features.iloc[0]['overview'])).unsqueeze(0).numpy()
    combined_features_matrix = combined_features

    cosine_similarities = cosine_similarity(input_movie_vector, combined_features_matrix).flatten()
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]

    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]

    if similar_movies.empty:
        return "No recommendations found."

    return similar_movies['title'].tolist()


In [ ]:
def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)

    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)

    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            collaborative_recommendations = recommend_movies_collaborative(imdb_id)
            content_recommendations = recommend_movies_by_content(normalized_movie_name)
            
            # Combine recommendations (you can use weighted combination)
            combined_recommendations = set(collaborative_recommendations + content_recommendations)
            return list(combined_recommendations)
        else:
            return recommend_movies_by_content(normalized_movie_name)
    else:
        return recommend_movies_by_content(normalized_movie_name)


# new chapter

In [35]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from transformers import BertTokenizer, BertModel
import torch
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Normalize movie titles by removing years and converting to lowercase
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

# Add normalized titles to the dataset
movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    # Map the normalized title back to the original titles.
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def recommend_movies_by_content(genres, director, movie_name):
    # Normalize the movie name for matching
    normalized_movie_name = normalize_title(movie_name)
    
    # Ensure director is a string
    if director is None:
        director = ''
    
    # Preprocess overviews and create BERT embeddings
    movies_with_tags['overview_bert'] = movies_with_tags['overview'].fillna('').apply(lambda x: bert_embedding(x))
    
    # Combine overview embeddings with TF-IDF for genres and director
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    genre_director_tfidf = tfidf_vectorizer.fit_transform(movies_with_tags['genre'].fillna('') + ' ' + movies_with_tags['director'].fillna(''))
    
    # Combine BERT embeddings and TF-IDF vectors
    combined_features = torch.cat((torch.tensor(movies_with_tags['overview_bert'].tolist()), 
                                   torch.tensor(genre_director_tfidf.toarray())), dim=1).numpy()
    
    # Vectorize the input movie's features
    input_movie_features = ' '.join([f"genre: {', '.join(genres)}", f"director: {director}"])
    input_movie_vector = torch.cat((torch.tensor(bert_embedding('')), 
                                    tfidf_vectorizer.transform([input_movie_features]).toarray()), dim=1).numpy()
    
    # Compute cosine similarity between the input movie and dataset
    cosine_similarities = cosine_similarity(input_movie_vector, combined_features).flatten()
    
    # Get indices of the top 10 most similar movies
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]
    
    # Debugging output
    print("Cosine similarities:", cosine_similarities)
    print("Similar indices:", similar_indices)
    
    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]
    
    # Check if there are any movies left to recommend
    if similar_movies.empty:
        return "No recommendations found."

    return similar_movies['title'].tolist()

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies(movie_name):
    # Normalize the input movie name
    normalized_movie_name = normalize_title(movie_name)
    
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        # Collaborative filtering
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            collaborative_recommendations = recommend_movies_collaborative(imdb_id)
            content_recommendations = recommend_movies_by_content(genres, director, normalized_movie_name)
            
            # Combine recommendations (you can use weighted combination)
            combined_recommendations = set(collaborative_recommendations.split('\n') + content_recommendations)
            return list(combined_recommendations)
        else:
            # Content-based filtering
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        # Fallback content-based filtering if IMDb ID is not available
        return recommend_movies_by_content(genres, director, normalized_movie_name)

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: None, IMDb ID: tt0113277


/tmp/ipykernel_36/4235241570.py:88: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  combined_features = torch.cat((torch.tensor(movies_with_tags['overview_bert'].tolist()),


TypeError: expected Tensor as element 1 in argument 0, but got numpy.ndarray

In [36]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from transformers import BertTokenizer, BertModel
import torch
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Normalize movie titles by removing years and converting to lowercase
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

# Add normalized titles to the dataset
movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    # Map the normalized title back to the original titles.
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    # Fetch movie details
    movie_details = movie.details(tmdb_id)
    
    # Fetch external IDs
    external_ids = movie.external_ids(tmdb_id)
    
    # Extract genres
    genres = [g['name'] for g in movie_details.get('genres', [])]
    
    # Extract the director from the credits
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    
    # Retrieve IMDb ID
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    
    return genres, director, imdb_id

def bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def recommend_movies_by_content(genres, director, movie_name):
    # Normalize the movie name for matching
    normalized_movie_name = normalize_title(movie_name)
    
    # Ensure director is a string
    if director is None:
        director = ''
    
    # Preprocess overviews and create BERT embeddings
    movies_with_tags['overview_bert'] = movies_with_tags['overview'].fillna('').apply(lambda x: bert_embedding(x))
    
    # Combine overview embeddings with TF-IDF for genres and director
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    genre_director_tfidf = tfidf_vectorizer.fit_transform(movies_with_tags['genre'].fillna('') + ' ' + movies_with_tags['director'].fillna(''))
    
    # Convert BERT embeddings to NumPy arrays
    overview_bert_array = np.array(movies_with_tags['overview_bert'].tolist())
    
    # Combine BERT embeddings and TF-IDF vectors
    combined_features = np.hstack((overview_bert_array, genre_director_tfidf.toarray()))
    
    # Vectorize the input movie's features
    input_movie_features = ' '.join([f"genre: {', '.join(genres)}", f"director: {director}"])
    input_movie_vector = np.hstack((bert_embedding(''), tfidf_vectorizer.transform([input_movie_features]).toarray()))
    
    # Compute cosine similarity between the input movie and dataset
    cosine_similarities = cosine_similarity(input_movie_vector.reshape(1, -1), combined_features).flatten()
    
    # Get indices of the top 10 most similar movies
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]
    
    # Debugging output
    print("Cosine similarities:", cosine_similarities)
    print("Similar indices:", similar_indices)
    
    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]
    
    # Check if there are any movies left to recommend
    if similar_movies.empty:
        return "No recommendations found."

    return similar_movies['title'].tolist()


def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies(movie_name):
    # Normalize the input movie name
    normalized_movie_name = normalize_title(movie_name)
    
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        # Collaborative filtering
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            collaborative_recommendations = recommend_movies_collaborative(imdb_id)
            content_recommendations = recommend_movies_by_content(genres, director, normalized_movie_name)
            
            # Combine recommendations (you can use weighted combination)
            combined_recommendations = set(collaborative_recommendations.split('\n') + content_recommendations)
            return list(combined_recommendations)
        else:
            # Content-based filtering
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        # Fallback content-based filtering if IMDb ID is not available
        return recommend_movies_by_content(genres, director, normalized_movie_name)

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please u

Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Genres: ['Action', 'Crime', 'Drama'], Director: None, IMDb ID: tt0113277


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))

In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie.external_ids(tmdb_id)
    genres = [g['name'] for g in movie_details.get('genres', [])]
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    return genres, director, imdb_id

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, movie_name):
    normalized_movie_name = normalize_title(movie_name)
    if director is None:
        director = ''
    
    # Extract BERT embeddings for the overview
    movie_overviews = movies_with_tags['overview'].fillna('')
    overview_embeddings = get_bert_embeddings(movie_overviews.tolist())

    # TF-IDF Vectorization for genres and director
    tfidf_vectorizer_genre = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    genre_tfidf_matrix = tfidf_vectorizer_genre.fit_transform(movies_with_tags['genre'].fillna(''))
    
    tfidf_vectorizer_director = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    director_tfidf_matrix = tfidf_vectorizer_director.fit_transform(movies_with_tags['director'].fillna(''))

    input_movie_features = {
        'genre': ' '.join([f"genre: {', '.join(genres)}"]),
        'director': f"director: {director}"
    }

    # TF-IDF Vectorization for the input movie genres and director
    input_genre_vector = tfidf_vectorizer_genre.transform([input_movie_features['genre']])
    input_director_vector = tfidf_vectorizer_director.transform([input_movie_features['director']])
    
    # Compute cosine similarity for genre and director
    genre_cosine_similarities = cosine_similarity(input_genre_vector, genre_tfidf_matrix).flatten()
    director_cosine_similarities = cosine_similarity(input_director_vector, director_tfidf_matrix).flatten()
    
    # Compute final similarity scores
    # Normalize to avoid bias
    genre_sim = (genre_cosine_similarities - genre_cosine_similarities.min()) / (genre_cosine_similarities.max() - genre_cosine_similarities.min())
    director_sim = (director_cosine_similarities - director_cosine_similarities.min()) / (director_cosine_similarities.max() - director_cosine_similarities.min())

    combined_similarities = genre_sim * 0.5 + director_sim * 0.5

    # Get indices of the top 10 most similar movies
    similar_indices = combined_similarities.argsort()[-11:-1][::-1]

    # Debugging output
    print("Cosine similarities (genre):", genre_cosine_similarities)
    print("Cosine similarities (director):", director_cosine_similarities)
    print("Combined similarities:", combined_similarities)
    print("Similar indices:", similar_indices)

    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]

    if similar_movies.empty:
        return "No recommendations found."

    recommended_movies = similar_movies['title'].tolist()
    
    if not recommended_movies:
        return "No recommendations found."
    
    return recommended_movies

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie.external_ids(tmdb_id)
    genres = [g['name'] for g in movie_details.get('genres', [])]
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    return genres, director, imdb_id

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings_filtered = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings_filtered.empty:
        return "No ratings data available for collaborative filtering."
    user_movie_ratings_filtered = user_movie_ratings_filtered.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings_filtered.pivot(index='userId', columns='title', values='rating').fillna(0)
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, movie_name):
    normalized_movie_name = normalize_title(movie_name)
    if director is None:
        director = ''
    
    # Extract BERT embeddings for the overview
    movie_overviews = movies_with_tags['overview'].fillna('')
    overview_embeddings = get_bert_embeddings(movie_overviews.tolist())

    # TF-IDF Vectorization for genres and director
    tfidf_vectorizer_genre = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    genre_tfidf_matrix = tfidf_vectorizer_genre.fit_transform(movies_with_tags['genre'].fillna(''))
    
    tfidf_vectorizer_director = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    director_tfidf_matrix = tfidf_vectorizer_director.fit_transform(movies_with_tags['director'].fillna(''))

    input_movie_features = {
        'genre': ' '.join([f"genre: {', '.join(genres)}"]),
        'director': f"director: {director}"
    }

    # TF-IDF Vectorization for the input movie genres and director
    input_genre_vector = tfidf_vectorizer_genre.transform([input_movie_features['genre']])
    input_director_vector = tfidf_vectorizer_director.transform([input_movie_features['director']])
    
    # Compute cosine similarity for genre and director
    genre_cosine_similarities = cosine_similarity(input_genre_vector, genre_tfidf_matrix).flatten()
    director_cosine_similarities = cosine_similarity(input_director_vector, director_tfidf_matrix).flatten()
    
    # Compute final similarity scores
    # Normalize to avoid bias
    genre_sim = (genre_cosine_similarities - genre_cosine_similarities.min()) / (genre_cosine_similarities.max() - genre_cosine_similarities.min())
    director_sim = (director_cosine_similarities - director_cosine_similarities.min()) / (director_cosine_similarities.max() - director_cosine_similarities.min())

    combined_similarities = genre_sim * 0.5 + director_sim * 0.5

    # Get indices of the top 10 most similar movies
    similar_indices = combined_similarities.argsort()[-11:-1][::-1]

    # Debugging output
    print("Cosine similarities (genre):", genre_cosine_similarities)
    print("Cosine similarities (director):", director_cosine_similarities)
    print("Combined similarities:", combined_similarities)
    print("Similar indices:", similar_indices)

    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]

    if similar_movies.empty:
        return "No recommendations found."

    recommended_movies = similar_movies['title'].tolist()
    
    if not recommended_movies:
        return "No recommendations found."
    
    return recommended_movies

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Normalize movie titles by removing years and converting to lowercase
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

# Add normalized titles to the dataset
movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie.external_ids(tmdb_id)
    genres = [g['name'] for g in movie_details.get('genres', [])]
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    return genres, director, imdb_id

def get_bert_embeddings(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = bert_model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

def compute_tfidf_features(texts, ngram_range=(1, 2)):
    vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=ngram_range)
    return vectorizer.fit_transform(texts)

def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")

    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings_filtered = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    
    if user_movie_ratings_filtered.empty:
        return "No ratings data available for collaborative filtering."
    
    user_movie_ratings_filtered = user_movie_ratings_filtered.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    
    user_movie_matrix = user_movie_ratings_filtered.pivot(index='userId', columns='title', values='rating').fillna(0)
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, movie_name):
    normalized_movie_name = normalize_title(movie_name)
    
    if director is None:
        director = ''
    
    # Generate BERT embeddings for overviews
    movie_overviews = movies_with_tags['overview'].fillna('')
    overview_embeddings = get_bert_embeddings(movie_overviews.tolist())
    
    # Compute TF-IDF features for genres and directors
    genre_tfidf_matrix = compute_tfidf_features(movies_with_tags['genre'].fillna(''))
    director_tfidf_matrix = compute_tfidf_features(movies_with_tags['director'].fillna(''))
    
    # Combine BERT embeddings and TF-IDF vectors
    combined_features = np.hstack((overview_embeddings, genre_tfidf_matrix.toarray(), director_tfidf_matrix.toarray()))
    
    # Compute BERT embedding for the input movie
    input_movie_features = ' '.join([f"genre: {', '.join(genres)}", f"director: {director}"])
    input_movie_vector = get_bert_embeddings([input_movie_features]).flatten()
    
    # Compute cosine similarity
    cosine_similarities = cosine_similarity(input_movie_vector.reshape(1, -1), combined_features).flatten()
    
    # Get indices of the top 10 most similar movies
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]
    
    print("Cosine similarities:", cosine_similarities)
    print("Similar indices:", similar_indices)
    
    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]
    
    if similar_movies.empty:
        return "No recommendations found."

    recommended_movies = similar_movies['title'].tolist()
    
    return recommended_movies if recommended_movies else "No recommendations found."

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))

# more features

In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie_details.get('external_ids', {})
    genres = [g['name'] for g in movie_details.get('genres', [])]
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    cast = movie_details.get('credits', {}).get('cast', [])
    crew = movie_details.get('credits', {}).get('crew', [])       
    imdb_id = external_ids.get('imdb_id') if external_ids else None
    synopsis = movie_details.get('overview', '')
    ratings = movie_details.get('vote_average', 0)
    reviews = movie_details.get('reviews', {}).get('results', [])
    actors = [a['name'] for a in movie_details.get('credits', {}).get('cast', [])[:5]]  # Top 5 actors
    keywords = [k['name'] for k in movie_details.get('keywords', {}).get('keywords', [])]

    return genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")
    print(f"Synopsis: {synopsis}")
    print(f"Ratings: {ratings}")
    print(f"Reviews: {reviews}")
    print(f"Actors: {actors}")
    print(f"Keywords: {keywords}")

    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings.empty:
        return "No ratings data available for collaborative filtering."
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, movie_name):
    normalized_movie_name = normalize_title(movie_name)
    if director is None:
        director = ''
    
    # Extract BERT embeddings for the synopsis
    movie_synopses = movies_with_tags['overview'].fillna('')
    synopsis_embeddings = get_bert_embeddings(movie_synopses.tolist())

    # TF-IDF Vectorization for genres, director, actors, and keywords
    tfidf_vectorizer_genre = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    genre_tfidf_matrix = tfidf_vectorizer_genre.fit_transform(movies_with_tags['genre'].fillna(''))
    
    tfidf_vectorizer_director = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    director_tfidf_matrix = tfidf_vectorizer_director.fit_transform(movies_with_tags['director'].fillna(''))
    
    tfidf_vectorizer_actor = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    actor_tfidf_matrix = tfidf_vectorizer_actor.fit_transform(movies_with_tags['actors'].fillna(''))
    
    tfidf_vectorizer_keywords = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    keywords_tfidf_matrix = tfidf_vectorizer_keywords.fit_transform(movies_with_tags['keywords'].fillna(''))

    input_movie_features = {
        'genre': ' '.join([f"genre: {', '.join(genres)}"]),
        'director': f"director: {director}",
        'actors': ' '.join([f"actor: {', '.join(actors)}"]),
        'keywords': ' '.join([f"keyword: {', '.join(keywords)}"])
    }

    # TF-IDF Vectorization for the input movie features
    input_genre_vector = tfidf_vectorizer_genre.transform([input_movie_features['genre']])
    input_director_vector = tfidf_vectorizer_director.transform([input_movie_features['director']])
    input_actor_vector = tfidf_vectorizer_actor.transform([input_movie_features['actors']])
    input_keywords_vector = tfidf_vectorizer_keywords.transform([input_movie_features['keywords']])
    
    # Compute cosine similarity for genre, director, actors, and keywords
    genre_cosine_similarities = cosine_similarity(input_genre_vector, genre_tfidf_matrix).flatten()
    director_cosine_similarities = cosine_similarity(input_director_vector, director_tfidf_matrix).flatten()
    actor_cosine_similarities = cosine_similarity(input_actor_vector, actor_tfidf_matrix).flatten()
    keywords_cosine_similarities = cosine_similarity(input_keywords_vector, keywords_tfidf_matrix).flatten()
    
    # Compute final similarity scores
    # Normalize to avoid bias
    genre_sim = (genre_cosine_similarities - genre_cosine_similarities.min()) / (genre_cosine_similarities.max() - genre_cosine_similarities.min())
    director_sim = (director_cosine_similarities - director_cosine_similarities.min()) / (director_cosine_similarities.max() - director_cosine_similarities.min())
    actor_sim = (actor_cosine_similarities - actor_cosine_similarities.min()) / (actor_cosine_similarities.max() - actor_cosine_similarities.min())
       # Continue with similarity normalization and combination
    keywords_sim = (keywords_cosine_similarities - keywords_cosine_similarities.min()) / (keywords_cosine_similarities.max() - keywords_cosine_similarities.min())

    # Combine the similarity scores
    combined_similarities = (genre_sim * 0.25 +
                             director_sim * 0.25 +
                             actor_sim * 0.25 +
                             keywords_sim * 0.25)

    # Get indices of the top 10 most similar movies
    similar_indices = combined_similarities.argsort()[-11:-1][::-1]

    # Debugging output
    print("Cosine similarities (genre):", genre_cosine_similarities)
    print("Cosine similarities (director):", director_cosine_similarities)
    print("Cosine similarities (actors):", actor_cosine_similarities)
    print("Cosine similarities (keywords):", keywords_cosine_similarities)
    print("Combined similarities:", combined_similarities)
    print("Similar indices:", similar_indices)

    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]

    if similar_movies.empty:
        return "No recommendations found."

    recommended_movies = similar_movies['title'].tolist()
    
    if not recommended_movies:
        return "No recommendations found."
    
    return recommended_movies

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie_details.get('external_ids', {})
    genres = [g['name'] for g in movie_details.get('genres', [])]
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    cast = movie_details.get('credits', {}).get('cast', [])[:5]  # Top 5 actors
    keywords = [k['name'] for k in movie_details.get('keywords', {}).get('keywords', [])]
    imdb_id = external_ids.get('imdb_id')
    synopsis = movie_details.get('overview', '')
    ratings = movie_details.get('vote_average', 0)
    reviews = movie_details.get('reviews', {}).get('results', [])

    return genres, director, imdb_id, synopsis, ratings, reviews, cast, keywords

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")
    print(f"Synopsis: {synopsis}")
    print(f"Ratings: {ratings}")
    print(f"Reviews: {reviews}")
    print(f"Actors: {actors}")
    print(f"Keywords: {keywords}")

    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings_subset = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings_subset.empty:
        return "No ratings data available for collaborative filtering."
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Apply SVD for collaborative filtering
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."
    
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, movie_name):
    normalized_movie_name = normalize_title(movie_name)
    if director is None:
        director = ''
    
    # Extract BERT embeddings for the synopsis
    movie_synopses = movies_with_tags['overview'].fillna('')
    synopsis_embeddings = get_bert_embeddings(movie_synopses.tolist())

    # TF-IDF Vectorization for genres, director, actors, and keywords
    tfidf_vectorizer_genre = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    genre_tfidf_matrix = tfidf_vectorizer_genre.fit_transform(movies_with_tags['genre'].fillna(''))
    
    tfidf_vectorizer_director = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    director_tfidf_matrix = tfidf_vectorizer_director.fit_transform(movies_with_tags['director'].fillna(''))
    
    tfidf_vectorizer_actor = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    actor_tfidf_matrix = tfidf_vectorizer_actor.fit_transform(movies_with_tags['cast'].fillna(''))
    
    tfidf_vectorizer_keywords = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    keywords_tfidf_matrix = tfidf_vectorizer_keywords.fit_transform(movies_with_tags['tag'].fillna(''))

    input_movie_features = {
        'genre': ' '.join([f"genre: {', '.join(genres)}"]),
        'director': f"director: {director}",
        'actors': ' '.join([f"actor: {', '.join([actor['name'] for actor in actors])}"]),
        'keywords': ' '.join([f"keyword: {', '.join(keywords)}"])
    }

    # TF-IDF Vectorization for the input movie features
    input_genre_vector = tfidf_vectorizer_genre.transform([input_movie_features['genre']])
    input_director_vector = tfidf_vectorizer_director.transform([input_movie_features['director']])
    input_actor_vector = tfidf_vectorizer_actor.transform([input_movie_features['actors']])
    input_keywords_vector = tfidf_vectorizer_keywords.transform([input_movie_features['keywords']])
    
    # Compute cosine similarity for genre, director, actors, and keywords
    genre_cosine_similarities = cosine_similarity(input_genre_vector, genre_tfidf_matrix).flatten()
    director_cosine_similarities = cosine_similarity(input_director_vector, director_tfidf_matrix).flatten()
    actor_cosine_similarities = cosine_similarity(input_actor_vector, actor_tfidf_matrix).flatten()
    keywords_cosine_similarities = cosine_similarity(input_keywords_vector, keywords_tfidf_matrix).flatten()
    
    # Normalize to avoid bias and compute final similarity scores
    genre_sim = (genre_cosine_similarities - genre_cosine_similarities.min()) / (genre_cosine_similarities.max() - genre_cosine_similarities.min())
    director_sim = (director_cosine_similarities - director_cosine_similarities.min()) / (director_cosine_similarities.max() - director_cosine_similarities.min())
    actor_sim = (actor_cosine_similarities - actor_cosine_similarities.min()) / (actor_cosine_similarities.max() - actor_cosine_similarities.min())
    keywords_sim = (keywords_cosine_similarities - keywords_cosine_similarities.min()) / (keywords_cosine_similarities.max() - keywords_cosine_similarities.min())

    # Combine the similarity scores
    combined_similarities = (genre_sim * 0.25 + director_sim * 0.25 + actor_sim * 0.25 + keywords_sim * 0.25)
    similar_movies_idx = np.argsort(combined_similarities)[::-1][:10]

    recommended_movies = []
    for idx in similar_movies_idx[:3]:
        similar_movie_title = movies_with_tags.iloc[idx]['title']
        if normalize_title(similar_movie_title) != normalized_movie_name:
            recommended_movies.append(f"Recommendation: {denormalize_title(similar_movie_title)}")

    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Example usage
recommended_movies = recommend_movies('Heat')
print(recommended_movies)


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Handle NA values
movies_with_tags.fillna({'genre': 'unknown', 'director': 'unknown', 'overview': 'No overview available', 
                         'cast': 'unknown', 'tag': 'no keywords'}, inplace=True)

def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie_details.get('external_ids', {})
    genres = [g['name'] for g in movie_details.get('genres', [])] or ['unknown']
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    director = director or 'unknown'
    cast = movie_details.get('credits', {}).get('cast', [])[:5] or ['unknown']
    keywords = [k['name'] for k in movie_details.get('keywords', {}).get('keywords', [])] or ['no keywords']
    imdb_id = external_ids.get('imdb_id')
    synopsis = movie_details.get('overview', 'No overview available')
    ratings = movie_details.get('vote_average', 0)
    reviews = movie_details.get('reviews', {}).get('results', [])

    return genres, director, imdb_id, synopsis, ratings, reviews, cast, keywords

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")
    print(f"Synopsis: {synopsis}")
    print(f"Ratings: {ratings}")
    print(f"Reviews: {reviews}")
    print(f"Actors: {actors}")
    print(f"Keywords: {keywords}")

    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)

def recommend_movies_collaborative(imdb_id):
    user_movie_ratings_subset = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings_subset.empty:
        return "No ratings data available for collaborative filtering."
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Apply SVD for collaborative filtering
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."
    
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

def recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, movie_name):
    normalized_movie_name = normalize_title(movie_name)
    
    # Extract BERT embeddings for the synopsis
    movie_synopses = movies_with_tags['overview'].fillna('')
    synopsis_embeddings = get_bert_embeddings(movie_synopses.tolist())

    # TF-IDF Vectorization for genres, director, actors, and keywords
    tfidf_vectorizer_genre = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    genre_tfidf_matrix = tfidf_vectorizer_genre.fit_transform(movies_with_tags['genre'])
    
    tfidf_vectorizer_director = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    director_tfidf_matrix = tfidf_vectorizer_director.fit_transform(movies_with_tags['director'])
    
    tfidf_vectorizer_actor = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    actor_tfidf_matrix = tfidf_vectorizer_actor.fit_transform(movies_with_tags['cast'])
    
    tfidf_vectorizer_keywords = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    keywords_tfidf_matrix = tfidf_vectorizer_keywords.fit_transform(movies_with_tags['tag'])

    input_movie_features = {
        'genre': ' '.join([f"genre: {', '.join(genres)}"]),
        'director': f"director: {director}",
        'actors': ' '.join([f"actor: {', '.join([actor['name'] for actor in actors])}"]),
        'keywords': ' '.join([f"keyword: {', '.join(keywords)}"])
    }

    # TF-IDF Vectorization for the input movie features
    input_genre_vector = tfidf_vectorizer_genre.transform([input_movie_features['genre']])
    input_director_vector = tfidf_vectorizer_director.transform([input_movie_features['director']])
    input_actor_vector = tfidf_vectorizer_actor.transform([input_movie_features['actors']])
    input_keywords_vector = tfidf_vectorizer_keywords.transform([input_movie_features['keywords']])
    
    # Compute cosine similarity for genre, director, actors, and keywords
    genre_cosine_similarities = cosine_similarity(input_genre_vector, genre_tfidf_matrix).flatten()
    director_cosine_similarities = cosine_similarity(input_director_vector, director_tfidf_matrix).flatten()
    actor_cosine_similarities = cosine_similarity(input_actor_vector, actor_tfidf_matrix).flatten()
    keywords_cosine_similarities = cosine_similarity(input_keywords_vector, keywords_tfidf_matrix).flatten()
    
    # Normalize to avoid bias and compute final similarity scores
    genre_sim = (genre_cosine_similarities - genre_cosine_similarities.min()) / (genre_cosine_similarities.max() - genre_cosine_similarities.min())
    director_sim = (director_cosine_similarities - director_cosine_similarities.min()) / (director_cosine_similarities.max() - director_cosine_similarities.min())
    actor_sim = (actor_cosine_similarities - actor_cosine_similarities.min()) / (actor_cosine_similarities.max() - actor_cosine_similarities.min())
    keywords_sim = (keywords_cosine_similarities - keywords_cosine_similarities.min()) / (keywords_cosine_similarities.max() - keywords_cosine_similarities.min())
    
    # Weighted sum of similarities
    combined_similarity = genre_sim * 0.3 + director_sim * 0.2 + actor_sim * 0.25 + keywords_sim * 0.25
    top_similar_movies_idx = combined_similarity.argsort()[::-1][:3]
    
    recommended_movies = []
    for idx in top_similar_movies_idx:
        recommended_movie = denormalize_title(movies_with_tags.iloc[idx]['normalized_title'])
        if recommended_movie and recommended_movie != normalized_movie_name:
            recommended_movies.append(recommended_movie)
    
    return recommended_movies if recommended_movies else "No recommendations found."

# Example usage
print(recommend_movies('Interstellar'))


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Handle NA values
movies_with_tags.fillna({'genre': 'unknown', 'director': 'unknown', 'overview': 'No overview available', 
                         'cast': 'unknown', 'tag': 'no keywords'}, inplace=True)

# Normalize movie titles
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

# Denormalize movie titles to original form
def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

# Fetch movie details using TMDb
def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie_details.get('external_ids', {})
    genres = [g['name'] for g in movie_details.get('genres', [])] or ['unknown']
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    director = director or 'unknown'
    cast = movie_details.get('credits', {}).get('cast', [])[:5] or ['unknown']
    keywords = [k['name'] for k in movie_details.get('keywords', {}).get('keywords', [])] or ['no keywords']
    imdb_id = external_ids.get('imdb_id')
    synopsis = movie_details.get('overview', 'No overview available')
    ratings = movie_details.get('vote_average', 0)
    reviews = movie_details.get('reviews', {}).get('results', [])

    return genres, director, imdb_id, synopsis, ratings, reviews, cast, keywords

# BERT embeddings for synopsis
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Main recommendation function
def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
    
    genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords = get_movie_details_by_tmdb(tmdb_id)
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")
    print(f"Synopsis: {synopsis}")
    print(f"Ratings: {ratings}")
    print(f"Reviews: {reviews}")
    print(f"Actors: {actors}")
    print(f"Keywords: {keywords}")

    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)

# Collaborative filtering recommendation
def recommend_movies_collaborative(imdb_id):
    user_movie_ratings_subset = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings_subset.empty:
        return "No ratings data available for collaborative filtering."
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Apply SVD for collaborative filtering
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)
    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."
    
    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Content-based recommendation
def recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, movie_name):
    normalized_movie_name = normalize_title(movie_name)
    
    # Extract BERT embeddings for the synopsis
    movie_synopses = movies_with_tags['overview'].fillna('')
    synopsis_embeddings = get_bert_embeddings(movie_synopses.tolist())

    # TF-IDF Vectorization for genres, director, actors, and keywords
    tfidf_vectorizer_genre = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    genre_tfidf_matrix = tfidf_vectorizer_genre.fit_transform(movies_with_tags['genre'])
    
    tfidf_vectorizer_director = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    director_tfidf_matrix = tfidf_vectorizer_director.fit_transform(movies_with_tags['director'])
    
    tfidf_vectorizer_actor = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    actor_tfidf_matrix = tfidf_vectorizer_actor.fit_transform(movies_with_tags['cast'])
    
    tfidf_vectorizer_keywords = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2))
    keywords_tfidf_matrix = tfidf_vectorizer_keywords.fit_transform(movies_with_tags['tag'])

    input_movie_features = {
        'genre': ' '.join([f"genre: {', '.join(genres)}"]),
        'director': f"director: {director}",
        'actors': ' '.join([f"actor: {', '.join([actor['name'] for actor in actors])}"]),
        'keywords': ' '.join([f"keyword: {', '.join(keywords)}"])
    }

    # TF-IDF Vectorization for the input movie features
    input_genre_vector = tfidf_vectorizer_genre.transform([input_movie_features['genre']])
    input_director_vector = tfidf_vectorizer_director.transform([input_movie_features['director']])
    input_actor_vector = tfidf_vectorizer_actor.transform([input_movie_features['actors']])
    input_keywords_vector = tfidf_vectorizer_keywords.transform([input_movie_features['keywords']])
    
    # Compute cosine similarity for genre, director, actors, and keywords
    genre_cosine_similarities = cosine_similarity(input_genre_vector, genre_tfidf_matrix).flatten()
    director_cosine_similarities = cosine_similarity(input_director_vector, director_tfidf_matrix).flatten()
    actor_cosine_similarities = cosine_similarity(input_actor_vector, actor_tfidf_matrix).flatten()
    keywords_cosine_similarities = cosine_similarity(input_keywords_vector, keywords_tfidf_matrix).flatten()
    
    # Normalize to avoid bias and compute final similarity scores
    genre_sim = (genre_cosine_similarities - genre_cosine_similarities.min()) / (genre_cosine_similarities.max() - genre_cosine_similarities.min())
    director_sim = (director_cosine_similarities - director_cosine_similarities.min()) / (director_cosine_similarities.max() - director_cosine_similarities.min())
    actor_sim = (actor_cosine_similarities - actor_cosine_similarities.min()) / (actor_cosine_similarities.max() - actor_cosine_similarities.min())
    keywords_sim = (keywords_cosine_similarities - keywords_cosine_similarities.min()) / (keywords_cosine_similarities.max() - keywords_cosine_similarities.min())

    # Combine all similarity scores (tune weights if needed)
    combined_similarities = 0.3 * genre_sim + 0.3 * director_sim + 0.2 * actor_sim + 0.2 * keywords_sim
    
    # Exclude the input movie from the recommendations
    indices = movies_with_tags[movies_with_tags['normalized_title'] != normalized_movie_name].index
    best_matches = indices[combined_similarities.argsort()[-3:][::-1]]

    recommended_movies = []
    for i, idx in enumerate(best_matches):
        recommended_movies.append(f"{i+1}. {movies_with_tags.loc[idx, 'title']}")

    return "\n".join(recommended_movies)

# Test the function with a sample movie name
movie_name = "Heat"
recommendations = recommend_movies(movie_name)
print(f"Recommendations for {movie_name}: \n{recommendations}")


# final trial

In [4]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Handle NA values
movies_with_tags.fillna({'genre': 'unknown', 'director': 'unknown', 'overview': 'No overview available', 
                         'cast': 'unknown', 'tag': 'no keywords'}, inplace=True)

# Normalize movie titles
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

# Denormalize movie titles to original form
def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

# Fetch movie details using TMDb
def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie_details.get('external_ids', {})
    genres = [g['name'] for g in movie_details.get('genres', [])] or ['unknown']
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    director = director or 'unknown'
    cast = movie_details.get('credits', {}).get('cast', [])[:5] or ['unknown']
    keywords = [k['name'] for k in movie_details.get('keywords', {}).get('keywords', [])] or ['no keywords']
    imdb_id = external_ids.get('imdb_id')
    synopsis = movie_details.get('overview', 'No overview available')
    ratings = movie_details.get('vote_average', 0)
    reviews = movie_details.get('reviews', {}).get('results', [])

    return genres, director, imdb_id, synopsis, ratings, reviews, cast, keywords

# BERT embeddings for synopsis
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Main recommendation function
def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
   
    genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords = get_movie_details_by_tmdb(tmdb_id)
    """
    print(f"Genres: {genres}, Director: {director}, IMDb ID: {imdb_id}")
    print(f"Synopsis: {synopsis}")
    print(f"Ratings: {ratings}")
    print(f"Reviews: {reviews}")
    print(f"Actors: {actors}")
    print(f"Keywords: {keywords}")
    """
    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)

# Collaborative filtering recommendation
def recommend_movies_collaborative(imdb_id):
    user_movie_ratings_subset = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings_subset.empty:
        return "No ratings data available for collaborative filtering."
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Apply SVD for collaborative filtering
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)

    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."
    # Content-based recommendation
def recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name):
    # Preprocess overviews and create BERT embeddings
    movies_with_tags['overview_bert'] = movies_with_tags['overview'].apply(get_bert_embeddings)

    # Combine BERT embeddings with TF-IDF for genres and director
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    genre_director_tfidf = tfidf_vectorizer.fit_transform(movies_with_tags['genre'] + ' ' + movies_with_tags['director'])

    # Convert BERT embeddings to NumPy arrays
    overview_bert_array = np.array(movies_with_tags['overview_bert'].tolist())

    # Reshape overview_bert_array from (10548, 1, 768) to (10548, 768)
    overview_bert_array = overview_bert_array.reshape(overview_bert_array.shape[0], -1)

    # Check the shapes
    num_bert_features = overview_bert_array.shape[1]
    num_tfidf_features = genre_director_tfidf.shape[1]

    print("Shape of overview_bert_array:", overview_bert_array.shape)
    print("Shape of genre_director_tfidf.toarray():", genre_director_tfidf.toarray().shape)

    if num_bert_features > num_tfidf_features:
        # Reduce the larger array (overview_bert_array) to match the smaller one (genre_director_tfidf)
        overview_bert_array = overview_bert_array[:, :num_tfidf_features]
    elif num_tfidf_features > num_bert_features:
        # Reduce the larger array (genre_director_tfidf) to match the smaller one (overview_bert_array)
        genre_director_tfidf = genre_director_tfidf[:, :num_bert_features]

    # Ensure both arrays are 2D and concatenate them
    combined_features = np.hstack((overview_bert_array, genre_director_tfidf.toarray()))

    # Vectorize the input movie's features
    input_movie_features = ' '.join([f"genre: {', '.join(genres)}", f"director: {director}"])
    input_movie_vector = np.hstack((get_bert_embeddings(synopsis).reshape(1, -1), tfidf_vectorizer.transform([input_movie_features]).toarray()))

    # Compute cosine similarity
    cosine_similarities = cosine_similarity(input_movie_vector, combined_features).flatten()

    # Get top similar movies
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]

    # Filter out the input movie itself
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]
    
    if similar_movies.empty:
        return "No recommendations found."
    
    return similar_movies['title'].tolist()

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please u

Enter a movie name:  heat


Found movie: Heat (TMDb ID: 949)
Shape of overview_bert_array: (10548, 768)
Shape of genre_director_tfidf.toarray(): (10548, 5186)


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 5954 while Y.shape[1] == 1536

In [5]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Handle NA values
movies_with_tags.fillna({'genre': 'unknown', 'director': 'unknown', 'overview': 'No overview available', 
                         'cast': 'unknown', 'tag': 'no keywords'}, inplace=True)

# Normalize movie titles
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

# Denormalize movie titles to original form
def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

# Fetch movie details using TMDb
def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie_details.get('external_ids', {})
    genres = [g['name'] for g in movie_details.get('genres', [])] or ['unknown']
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    director = director or 'unknown'
    cast = movie_details.get('credits', {}).get('cast', [])[:5] or ['unknown']
    keywords = [k['name'] for k in movie_details.get('keywords', {}).get('keywords', [])] or ['no keywords']
    imdb_id = external_ids.get('imdb_id')
    synopsis = movie_details.get('overview', 'No overview available')
    ratings = movie_details.get('vote_average', 0)
    reviews = movie_details.get('reviews', {}).get('results', [])

    return genres, director, imdb_id, synopsis, ratings, reviews, cast, keywords

# BERT embeddings for synopsis
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)  # Added padding/truncation
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Main recommendation function
def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
   
    genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords = get_movie_details_by_tmdb(tmdb_id)
    
    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)

# Collaborative filtering recommendation
def recommend_movies_collaborative(imdb_id):
    user_movie_ratings_subset = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings_subset.empty:
        return "No ratings data available for collaborative filtering."
    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Apply SVD for collaborative filtering
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)

    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Content-based recommendation
def recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name):
    # Preprocess overviews and create BERT embeddings
    movies_with_tags['overview_bert'] = movies_with_tags['overview'].apply(get_bert_embeddings)

    # Combine BERT embeddings with TF-IDF for genres and director
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    genre_director_tfidf = tfidf_vectorizer.fit_transform(movies_with_tags['genre'] + ' ' + movies_with_tags['director'])

    # Convert BERT embeddings to NumPy arrays
    overview_bert_array = np.array(movies_with_tags['overview_bert'].tolist())

    # Reshape overview_bert_array from (10548, 1, 768) to (10548, 768)
    overview_bert_array = overview_bert_array.reshape(overview_bert_array.shape[0], -1)

    # Check the shapes of BERT and TF-IDF feature arrays
    num_bert_features = overview_bert_array.shape[1]
    num_tfidf_features = genre_director_tfidf.shape[1]

    print("Shape of overview_bert_array:", overview_bert_array.shape)
    print("Shape of genre_director_tfidf.toarray():", genre_director_tfidf.toarray().shape)

    # Adjust dimensions by padding or trimming to ensure both have the same size
    if num_bert_features > num_tfidf_features:
        # If BERT features are larger, trim BERT array to match TF-IDF
        overview_bert_array = overview_bert_array[:, :num_tfidf_features]
    elif num_tfidf_features > num_bert_features:
        # If TF-IDF features are larger, pad TF-IDF array to match BERT
        padding_width = num_tfidf_features - num_bert_features
        padding = np.zeros((overview_bert_array.shape[0], padding_width))
        overview_bert_array = np.hstack((overview_bert_array, padding))

    # Combine the adjusted arrays
    combined_features = np.hstack((overview_bert_array, genre_director_tfidf.toarray()))

    # Vectorize the input movie's features (for new movie content)
    input_movie_features = ' '.join([f"genre: {', '.join(genres)}", f"director: {director}"])
    input_movie_vector = np.hstack((get_bert_embeddings(synopsis).reshape(1, -1), tfidf_vectorizer.transform([input_movie_features]).toarray()))

    # Ensure the input vector also matches the combined feature size
    if input_movie_vector.shape[1] < combined_features.shape[1]:
        padding_width = combined_features.shape[1] - input_movie_vector.shape[1]
        input_movie_vector = np.hstack((input_movie_vector, np.zeros((1, padding_width))))
    elif input_movie_vector.shape[1] > combined_features.shape[1]:
        input_movie_vector = input_movie_vector[:, :combined_features.shape[1]]

    # Compute cosine similarity
    cosine_similarities = cosine_similarity(input_movie_vector, combined_features).flatten()

    # Get top similar movies and filter out the input movie itself
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]

    # Return recommendations
    if similar_movies.empty:
        return "No recommendations found."
    else:
        return similar_movies['title'].tolist()

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please u

Enter a movie name:  me before you


Found movie: Me Before You (TMDb ID: 296096)
Shape of overview_bert_array: (10548, 768)
Shape of genre_director_tfidf.toarray(): (10548, 5186)
['The Kid', 'The Kid', 'I Capture the Castle', 'Stanley & Iris', 'Bella', 'The Girl Next Door', 'The Girl Next Door', 'Me and You and Everyone We Know', "Breakfast at Tiffany's", 'Man on the Train']


In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Handle NA values
movies_with_tags.fillna({'genre': 'unknown', 'director': 'unknown', 'overview': 'No overview available', 
                         'cast': 'unknown', 'tag': 'no keywords'}, inplace=True)

# Normalize movie titles
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

# Denormalize movie titles to original form
def denormalize_title(normalized_title):
    for original_title in movies_with_tags['title']:
        if normalize_title(original_title) == normalized_title:
            return original_title
    return None

# Fetch movie details using TMDb
def get_movie_details_by_tmdb(tmdb_id):
    movie_details = movie.details(tmdb_id)
    external_ids = movie_details.get('external_ids', {})
    genres = [g['name'] for g in movie_details.get('genres', [])] or ['unknown']
    director = None
    for crew_member in movie_details.get('credits', {}).get('crew', []):
        if crew_member['job'] == 'Director':
            director = crew_member['name']
            break
    director = director or 'unknown'
    cast = movie_details.get('credits', {}).get('cast', [])[:5] or ['unknown']
    keywords = [k['name'] for k in movie_details.get('keywords', {}).get('keywords', [])] or ['no keywords']
    imdb_id = external_ids.get('imdb_id')
    synopsis = movie_details.get('overview', 'No overview available')
    ratings = movie_details.get('vote_average', 0)
    reviews = movie_details.get('reviews', {}).get('results', [])

    return genres, director, imdb_id, synopsis, ratings, reviews, cast, keywords

# BERT embeddings for synopsis
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Main recommendation function
def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
   
    genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords = get_movie_details_by_tmdb(tmdb_id)
    
    if imdb_id:
        if imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
            return recommend_movies_collaborative(imdb_id)
        else:
            return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)
    else:
        return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)

# Collaborative filtering recommendation
def recommend_movies_collaborative(imdb_id):
    user_movie_ratings_subset = user_movie_ratings[user_movie_ratings['imdbId'] == imdb_id]
    if user_movie_ratings_subset.empty:
        return "No ratings data available for collaborative filtering."

    user_movie_ratings = user_movie_ratings.groupby(['userId', 'title'], as_index=False).agg({'rating': 'mean'})
    user_movie_matrix = user_movie_ratings.pivot(index='userId', columns='title', values='rating').fillna(0)

    # Apply SVD for collaborative filtering
    svd = TruncatedSVD(n_components=20)
    matrix_factorized = svd.fit_transform(user_movie_matrix)

    cosine_sim_matrix = cosine_similarity(matrix_factorized.T)
    
    try:
        movie_idx = list(user_movie_matrix.columns).index(imdb_id)
    except ValueError:
        return "Movie not found in the ratings dataset."

    similar_movies = list(enumerate(cosine_sim_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    recommended_movies = []
    for i, (movie_idx, score) in enumerate(similar_movies[:10]):
        similar_movie_title = user_movie_matrix.columns[movie_idx]
        recommended_movies.append(f"{i+1}. {similar_movie_title}")
        if len(recommended_movies) == 3:
            break
    
    return "\n".join(recommended_movies) if recommended_movies else "No recommendations found."

# Content-based recommendation
def recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name):
    # Preprocess overviews and create BERT embeddings
    movies_with_tags['overview_bert'] = movies_with_tags['overview'].apply(lambda x: get_bert_embeddings([x])[0])

    # Combine BERT embeddings with TF-IDF for genres and director
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    genre_director_tfidf = tfidf_vectorizer.fit_transform(movies_with_tags['genre'] + ' ' + movies_with_tags['director'])

    # Convert BERT embeddings to NumPy arrays
    overview_bert_array = np.array(movies_with_tags['overview_bert'].tolist())

    # Reshape overview_bert_array from (10548, 1, 768) to (10548, 768)
    overview_bert_array = overview_bert_array.reshape(overview_bert_array.shape[0], -1)

    # Check the shapes of BERT and TF-IDF feature arrays
    num_bert_features = overview_bert_array.shape[1]
    num_tfidf_features = genre_director_tfidf.shape[1]

    print("Shape of overview_bert_array:", overview_bert_array.shape)
    print("Shape of genre_director_tfidf.toarray():", genre_director_tfidf.toarray().shape)

    # Adjust dimensions by padding or trimming to ensure both have the same size
    if num_bert_features > num_tfidf_features:
        overview_bert_array = overview_bert_array[:, :num_tfidf_features]
    elif num_tfidf_features > num_bert_features:
        padding_width = num_tfidf_features - num_bert_features
        padding = np.zeros((overview_bert_array.shape[0], padding_width))
        overview_bert_array = np.hstack((overview_bert_array, padding))

    # Combine the adjusted arrays
    combined_features = np.hstack((overview_bert_array, genre_director_tfidf.toarray()))

    # Vectorize the input movie's features (for new movie content)
    input_movie_features = ' '.join([f"genre: {', '.join(genres)}", f"director: {director}"])
    input_movie_vector = np.hstack((get_bert_embeddings([synopsis])[0].reshape(1, -1), tfidf_vectorizer.transform([input_movie_features]).toarray()))

    # Ensure the input vector also matches the combined feature size
    if input_movie_vector.shape[1] < combined_features.shape[1]:
        padding_width = combined_features.shape[1] - input_movie_vector.shape[1]
        input_movie_vector = np.hstack((input_movie_vector, np.zeros((1, padding_width))))
    elif input_movie_vector.shape[1] > combined_features.shape[1]:
        input_movie_vector = input_movie_vector[:, :combined_features.shape[1]]

    # Compute cosine similarity
    cosine_similarities = cosine_similarity(input_movie_vector, combined_features).flatten()

    # Get top similar movies and filter out the input movie itself
    similar_indices = cosine_similarities.argsort()[-11:-1][::-1]
    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]

    # Return recommendations
    if similar_movies.empty:
        return "No recommendations found."
    else:
        return "\n".join([f"{i+1}. {row['title']}: {row['overview']}" for i, row in similar_movies.head(3).iterrows()])

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))


In [ ]:
# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))

# deployment version of the model 

In [ ]:
from tmdbv3api import TMDb, Movie
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import numpy as np
from sklearn.decomposition import TruncatedSVD
from joblib import Parallel, delayed
import joblib
import os

# Initialize TMDb
tmdb = TMDb()
tmdb.api_key = 'ddad317e776c8ec2f92ec52efe9d34f5'

# Set up Movie object
movie = Movie()

# Load the datasets
ratings_path = '/kaggle/input/rating-imdbid/ratings.csv'
user_movie_ratings = pd.read_csv(ratings_path)
movies_with_tags = pd.read_csv('/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Initialize DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Handle NA values
movies_with_tags.fillna({
    'genre': 'unknown',
    'director': 'unknown',
    'overview': 'No overview available',
    'cast': 'unknown',
    'tag': 'no keywords'
}, inplace=True)

# Normalize movie titles
def normalize_title(title):
    if pd.isna(title):
        return ''
    title = title.lower().strip()
    title = re.sub(r'\(\d{4}\)', '', title).strip()
    return title

movies_with_tags['normalized_title'] = movies_with_tags['title'].apply(normalize_title)

# Cache for TMDb API responses
tmdb_cache = {}

# Precompute movie embeddings and save them
def cache_tmdb_movie_details(tmdb_id):
    if tmdb_id not in tmdb_cache:
        movie_details = movie.details(tmdb_id)
        genres = [g['name'] for g in movie_details.get('genres', [])] or ['unknown']
        director = 'unknown'
        for crew_member in movie_details.get('credits', {}).get('crew', []):
            if crew_member['job'] == 'Director':
                director = crew_member['name']
                break
        cast = movie_details.get('credits', {}).get('cast', [])[:5] or ['unknown']
        keywords = [k['name'] for k in movie_details.get('keywords', {}).get('keywords', [])] or ['no keywords']
        synopsis = movie_details.get('overview', 'No overview available')
        ratings = movie_details.get('vote_average', 0)
        reviews = movie_details.get('reviews', {}).get('results', [])
        imdb_id = movie_details.get('external_ids', {}).get('imdb_id')

        tmdb_cache[tmdb_id] = (genres, director, imdb_id, synopsis, ratings, reviews, cast, keywords)
        return genres, director, imdb_id, synopsis, ratings, reviews, cast, keywords
    return tmdb_cache[tmdb_id]

# BERT embeddings for synopsis
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = distilbert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Precompute and store embeddings for movie overviews
def precompute_embeddings_and_similarity():
    # Precompute BERT embeddings
    if not os.path.exists("overview_bert_embeddings.npy"):
        print("Precomputing BERT embeddings for movie overviews...")
        bert_embeddings = Parallel(n_jobs=-1)(delayed(get_bert_embeddings)(overview) for overview in movies_with_tags['overview'])
        bert_embeddings = np.vstack(bert_embeddings)
        np.save("overview_bert_embeddings.npy", bert_embeddings)
    else:
        print("Loading precomputed BERT embeddings...")
        bert_embeddings = np.load("overview_bert_embeddings.npy")

    # Precompute TF-IDF vectors for genres and director
    if not os.path.exists("tfidf_features.npy"):
        print("Precomputing TF-IDF features for genres and directors...")
        tfidf_vectorizer = TfidfVectorizer(stop_words='english')
        genre_director_tfidf = tfidf_vectorizer.fit_transform(movies_with_tags['genre'] + ' ' + movies_with_tags['director'])
        joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
        np.save("tfidf_features.npy", genre_director_tfidf.toarray())
    else:
        print("Loading precomputed TF-IDF features...")
        genre_director_tfidf = np.load("tfidf_features.npy")
    
    # Reduce dimensions for faster similarity computations
    print("Reducing dimensions with TruncatedSVD...")
    svd = TruncatedSVD(n_components=100)
    bert_embeddings_reduced = svd.fit_transform(bert_embeddings)
    genre_director_tfidf_reduced = svd.fit_transform(genre_director_tfidf)

    # Compute cosine similarities between all movies and store them
    combined_features = np.hstack((bert_embeddings_reduced, genre_director_tfidf_reduced))
    similarity_matrix = cosine_similarity(combined_features)
    np.save("similarity_matrix.npy", similarity_matrix)

# Run precomputation process (will skip if already done)
precompute_embeddings_and_similarity()

# Main recommendation function using precomputed data
def recommend_movies(movie_name):
    normalized_movie_name = normalize_title(movie_name)
    search_results = movie.search(movie_name)
    
    if not search_results:
        return f"No movie found for '{movie_name}'"
    
    tmdb_id = search_results[0].id
    print(f"Found movie: {search_results[0].title} (TMDb ID: {tmdb_id})")
   
    genres, director, imdb_id, synopsis, ratings, reviews, actors, keywords = cache_tmdb_movie_details(tmdb_id)
    
    if imdb_id and imdb_id in user_movie_ratings['imdbId'].astype(str).unique():
        return recommend_movies_collaborative(imdb_id)
    else:
        return recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name)

# Content-based recommendation using precomputed similarity matrix
def recommend_movies_by_content(genres, director, synopsis, ratings, reviews, actors, keywords, normalized_movie_name):
    similarity_matrix = np.load("similarity_matrix.npy")
    movie_idx = movies_with_tags.index[movies_with_tags['normalized_title'] == normalized_movie_name].tolist()

    if not movie_idx:
        return f"No recommendations found for '{normalized_movie_name}'"
    
    movie_idx = movie_idx[0]
    similar_indices = similarity_matrix[movie_idx].argsort()[-11:-1][::-1]

    similar_movies = movies_with_tags.iloc[similar_indices]
    similar_movies = similar_movies[similar_movies['normalized_title'] != normalized_movie_name]

    if similar_movies.empty:
        return "No recommendations found."
    
    return similar_movies['title'].tolist()

# Example usage:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name))

In [ ]:
# Example saving files
np.save("/kaggle/working/overview_bert_embeddings.npy", bert_embeddings)
np.save("/kaggle/working/tfidf_features.npy", genre_director_tfidf)
np.save("/kaggle/working/similarity_matrix.npy", similarity_matrix)